In [1]:
import random
import warnings
from sklearn.utils import check_X_y, check_array


import numpy as np
import pandas as pd
from scipy.fft import fft
from scipy.optimize import minimize

# Sklearn Core & Metrics
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.calibration import CalibratedClassifierCV
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import (
    LinearDiscriminantAnalysis,
    QuadraticDiscriminantAnalysis,
)
from sklearn.ensemble import (
    ExtraTreesClassifier,
    RandomForestClassifier,
    HistGradientBoostingClassifier,
)
from sklearn.linear_model import RidgeClassifier
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import (
    StratifiedKFold,
    train_test_split,
    cross_val_predict,
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import (
    PowerTransformer,
    RobustScaler,
    StandardScaler,
    MinMaxScaler,
)
from sklearn.svm import SVC, NuSVC, LinearSVC
from sklearn.kernel_approximation import RBFSampler
from sklearn.random_projection import GaussianRandomProjection
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import log_loss, accuracy_score
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted

# Gradient Boosting
from xgboost import XGBClassifier

# GPU CHECK
try:
    import cupy as cp

    GPU_AVAILABLE = True
    print("✅ GPU DETECTED: HRF v26.0 'Holo-Fractal Universe' Active")
except ImportError:
    GPU_AVAILABLE = False
    print("⚠️ GPU NOT FOUND: Running in Slow Mode")

warnings.filterwarnings("ignore")


# --- 1. THE HOLOGRAPHIC SOUL (Unit 3 - Multiverse Edition - VRAM PINNED) ---
class HolographicSoulUnit(BaseEstimator, ClassifierMixin):
    def __init__(self, k=15):
        self.k = k
        self.dna_ = {
            "freq": 2.0, "gamma": 0.5, "power": 2.0,
            "metric": "minkowski", "p": 2.0, "phase": 0.0,
            "dim_reduction": "none",
        }
        self.projector_ = None
        self.X_raw_source_ = None
        # GPU Cache
        self._X_train_gpu = None
        self._y_train_gpu = None
        # Pre-calculated norms for fast Euclidean
        self._X_train_sq_norm = None

    def fit(self, X, y):
        self.classes_ = np.unique(y)
        self._apply_projection(X)
        self.y_train_ = y

        # [TITAN OPTIMIZATION] Upload to GPU ONCE
        if GPU_AVAILABLE:
            self._X_train_gpu = cp.asarray(self.X_train_, dtype=cp.float32)
            self._y_train_gpu = cp.asarray(self.y_train_)
            # Pre-calc Squared Norm for Fast Euclidean Path
            self._X_train_sq_norm = cp.sum(self._X_train_gpu ** 2, axis=1)

        return self

    def _apply_projection(self, X):
        if self.dna_["dim_reduction"] == "holo":
            n_components = max(2, int(np.sqrt(X.shape[1])))
            self.projector_ = GaussianRandomProjection(n_components=n_components, random_state=42)
            self.X_train_ = self.projector_.fit_transform(X)
        elif self.dna_["dim_reduction"] == "pca":
            n_components = max(2, int(np.sqrt(X.shape[1])))
            self.projector_ = PCA(n_components=n_components, random_state=42)
            self.X_train_ = self.projector_.fit_transform(X)
        else:
            self.projector_ = None
            self.X_train_ = X

    def set_raw_source(self, X):
        self.X_raw_source_ = X

    def evolve(self, X_val, y_val, generations=10):
        if not GPU_AVAILABLE: return 0.0

        # [TITAN OPTIMIZATION] Pre-load Validation Data
        X_val_curr = self.projector_.transform(X_val) if self.projector_ else X_val
        X_val_g = cp.asarray(X_val_curr, dtype=cp.float32)
        y_val_g = cp.asarray(y_val)

        # Pre-calc validation norm for Fast Euclidean
        val_sq_norm = cp.sum(X_val_g ** 2, axis=1)

        n_universes = 8 # Slightly reduced for speed, keeps high diversity
        best_dna = self.dna_.copy()

        # Smart Init (Fast Sample)
        sample_X = self._X_train_gpu[:100]
        dists = cp.mean(cp.linalg.norm(sample_X[:, None, :] - sample_X[None, :, :], axis=2))
        median_dist = float(cp.asnumpy(dists))
        if median_dist > 0: best_dna["freq"] = 3.14159 / median_dist

        # Initial Score
        best_acc = self._score_on_gpu(X_val_g, y_val_g, val_sq_norm)

        patience = 0

        for gen in range(generations):
            candidates = []
            for _ in range(n_universes):
                mutant = best_dna.copy()
                trait = random.choice(list(mutant.keys()))

                if trait == "freq": mutant["freq"] *= np.random.uniform(0.8, 1.25)
                elif trait == "gamma": mutant["gamma"] = np.random.uniform(0.1, 5.0)
                elif trait == "power": mutant["power"] = random.choice([0.5, 1.0, 2.0, 3.0, 4.0, 6.0])
                elif trait == "p":
                    # 50% chance to snap to 2.0 (Fast Path), 50% random
                    if random.random() < 0.5: mutant["p"] = 2.0
                    else: mutant["p"] = np.clip(mutant["p"] + np.random.uniform(-0.5, 0.5), 0.5, 8.0)
                elif trait == "phase": mutant["phase"] = np.random.uniform(0, 3.14159)
                candidates.append(mutant)

            generation_best_acc = -1
            generation_best_dna = None

            for mutant_dna in candidates:
                self.dna_ = mutant_dna
                # Score using fast internal method
                acc = self._score_on_gpu(X_val_g, y_val_g, val_sq_norm)

                if acc > generation_best_acc:
                    generation_best_acc = acc
                    generation_best_dna = mutant_dna

            if generation_best_acc >= best_acc:
                best_acc = generation_best_acc
                best_dna = generation_best_dna
                patience = 0
            else:
                patience += 1

            # Reset to best
            self.dna_ = best_dna

            # [TITAN OPTIMIZATION] Early Stopping
            # If we don't improve for 8 generations, the soul is mature.
            if patience >= 8:
                break

        self.dna_ = best_dna
        del X_val_g, y_val_g, val_sq_norm
        cp.get_default_memory_pool().free_all_blocks()

        return best_acc

    def _score_on_gpu(self, X_val_g, y_val_g, val_sq_norm=None):
        probs = self._predict_proba_gpu_internal(X_val_g, val_sq_norm)
        preds = cp.argmax(probs, axis=1)
        return float(cp.mean(preds == y_val_g))

    def predict_proba(self, X):
        if self.projector_ is not None: X_curr = self.projector_.transform(X)
        else: X_curr = X

        if GPU_AVAILABLE:
            X_g = cp.asarray(X_curr, dtype=cp.float32)
            # Calc Norm for new data
            x_sq_norm = cp.sum(X_g ** 2, axis=1)
            probs = self._predict_proba_gpu_internal(X_g, x_sq_norm)
            return cp.asnumpy(probs)
        else:
            return np.zeros((len(X), len(self.classes_)))

    def _predict_proba_gpu_internal(self, X_te_g, X_te_sq_norm=None):
        n_test = len(X_te_g)
        n_classes = len(self.classes_)
        probas = []
        # Increased Batch Size for T4 (Matrix Multiplication can handle it)
        #batch_size = 256 # for wide datasets
        batch_size = 2048

        p_norm = self.dna_.get("p", 2.0)
        gamma = self.dna_["gamma"]
        freq = self.dna_["freq"]
        power = self.dna_["power"]
        phase = self.dna_.get("phase", 0.0)

        # CHECK: Can we use Fast Euclidean? (p ~= 2.0)
        use_fast_path = abs(p_norm - 2.0) < 0.05

        for i in range(0, n_test, batch_size):
            end = min(i + batch_size, n_test)
            batch_te = X_te_g[i:end]

            # --- DISTANCE CALCULATION ---
            if use_fast_path and self._X_train_sq_norm is not None:
                # [FAST PATH] A^2 + B^2 - 2AB
                # 50x Speedup using Matrix Multiplication
                if X_te_sq_norm is not None:
                    batch_sq = X_te_sq_norm[i:end][:, None]
                else:
                    batch_sq = cp.sum(batch_te**2, axis=1, keepdims=True)

                train_sq = self._X_train_sq_norm[None, :]
                dot_prod = cp.dot(batch_te, self._X_train_gpu.T)

                dists_sq = batch_sq + train_sq - 2 * dot_prod
                dists_sq = cp.maximum(dists_sq, 0.0)
                dists = cp.sqrt(dists_sq)
            else:
                # [SLOW PATH] Broadcasting for non-Euclidean metrics (p != 2)
                diff = cp.abs(batch_te[:, None, :] - self._X_train_gpu[None, :, :])
                dists = cp.sum(cp.power(diff, p_norm), axis=2)
                dists = cp.power(dists, 1.0 / p_norm)

            # --- WEIGHTING (RESONANCE) ---
            # argpartition is faster than argsort for finding Top K
            top_k_idx = cp.argsort(dists, axis=1)[:, : self.k]

            row_idx = cp.arange(len(batch_te))[:, None]
            top_dists = dists[row_idx, top_k_idx]
            top_y = self._y_train_gpu[top_k_idx]

            cosine_term = 1.0 + cp.cos(freq * top_dists + phase)
            cosine_term = cp.maximum(cosine_term, 0.0)
            w = cp.exp(-gamma * (top_dists**2)) * cosine_term
            w = cp.power(w, power)

            batch_probs = cp.zeros((len(batch_te), n_classes))
            for c_idx, cls in enumerate(self.classes_):
                class_mask = top_y == cls
                batch_probs[:, c_idx] = cp.sum(w * class_mask, axis=1)

            total_energy = cp.sum(batch_probs, axis=1, keepdims=True)
            total_energy[total_energy == 0] = 1.0
            batch_probs /= total_energy
            probas.append(batch_probs)

        return cp.concatenate(probas)

    def predict(self, X):
        return self.classes_[np.argmax(self.predict_proba(X), axis=1)]

    def score(self, X, y):
        return accuracy_score(y, self.predict(X))


# --- 3. THE QUANTUM FIELD (Unit 4 - Reserve) ---
class QuantumFieldUnit(BaseEstimator, ClassifierMixin):
    def __init__(self):
        self.rbf_feature_ = RBFSampler(n_components=100, random_state=42)
        self.classifier_ = RidgeClassifier(alpha=1.0)
        self.classes_ = None
        self.dna_ = {"gamma": 1.0, "n_components": 100}

    def fit(self, X, y):
        self.classes_ = np.unique(y)
        self.rbf_feature_.set_params(
            gamma=self.dna_["gamma"], n_components=self.dna_["n_components"]
        )
        X_quantum = self.rbf_feature_.fit_transform(X)
        self.classifier_.fit(X_quantum, y)
        return self

    def predict_proba(self, X):
        X_quantum = self.rbf_feature_.transform(X)
        d = self.classifier_.decision_function(X_quantum)
        if len(self.classes_) == 2:
            probs = 1 / (1 + np.exp(-d))
            return np.column_stack([1 - probs, probs])
        else:
            exp_d = np.exp(d - np.max(d, axis=1, keepdims=True))
            return exp_d / np.sum(exp_d, axis=1, keepdims=True)

    def score(self, X, y):
        return accuracy_score(y, self.classes_[np.argmax(self.predict_proba(X), axis=1)])


# --- 4. THE ENTROPY MAXWELL (Unit 5 - Reserve) ---
class EntropyMaxwellUnit(BaseEstimator, ClassifierMixin):
    def __init__(self):
        self.models_ = {}
        self.classes_ = None
        self.priors_ = None
        self.dna_ = {"n_components": 1}

    def fit(self, X, y):
        self.classes_ = np.unique(y)
        self.models_ = {}
        self.priors_ = {}
        n_samples = len(y)
        for cls in self.classes_:
            X_c = X[y == cls]
            if len(X_c) < 2:
                self.priors_[cls] = 0.0
                continue
            self.priors_[cls] = len(X_c) / n_samples
            n_comp = min(self.dna_["n_components"], len(X_c))
            gmm = GaussianMixture(
                n_components=n_comp, covariance_type="full", reg_covar=1e-4, random_state=42
            )
            gmm.fit(X_c)
            self.models_[cls] = gmm
        return self

    def predict_proba(self, X):
        probs = np.zeros((len(X), len(self.classes_)))
        for i, cls in enumerate(self.classes_):
            if cls in self.models_:
                log_prob = self.models_[cls].score_samples(X)
                log_prob = np.clip(log_prob, -100, 100)
                probs[:, i] = np.exp(log_prob) * self.priors_[cls]
        total = np.sum(probs, axis=1, keepdims=True) + 1e-10
        return probs / total

    def score(self, X, y):
        return accuracy_score(y, self.classes_[np.argmax(self.predict_proba(X), axis=1)])


# --- 5. THE OMNI-KERNEL NEXUS (Unit 6 - Reserve) ---
class OmniKernelUnit(BaseEstimator, ClassifierMixin):
    def __init__(self):
        self.model_ = None
        self.classes_ = None
        self.dna_ = {
            "kernel": "rbf",
            "C": 1.0,
            "gamma": "scale",
            "degree": 3,
            "coef0": 0.0,
        }

    def fit(self, X, y):
        self.classes_ = np.unique(y)
        self.model_ = SVC(
            kernel=self.dna_["kernel"],
            C=self.dna_["C"],
            gamma=self.dna_["gamma"],
            degree=self.dna_["degree"],
            coef0=self.dna_["coef0"],
            probability=True,
            random_state=42,
            cache_size=500,
        )
        self.model_.fit(X, y)
        return self

    def predict_proba(self, X):
        return self.model_.predict_proba(X)

    def score(self, X, y):
        return self.model_.score(X, y)


# --- 18. THE GOLDEN SPIRAL (Unit 18 - Nature's Code) ---
# --- 18. THE GOLDEN FOREST (GPU T4 - Parallel Ensemble) ---
class GoldenSpiralUnit(BaseEstimator, ClassifierMixin):
    def __init__(self, k=21, n_estimators=100):
        # n_estimators=50 ensures 'Forest' power but keeps it sub-second on GPU
        self.k = k
        self.n_estimators = n_estimators
        self.classes_ = None
        self.X_train_ = None
        self.y_train_ = None
        # DNA: The "Seed" parameters for the forest
        self.dna_ = {"resonance": 1.618, "decay": 1.618, "shift": 137.5}

    def fit(self, X, y):
        self.classes_ = np.unique(y)
        if GPU_AVAILABLE:
            self.X_train_ = cp.asarray(X, dtype=cp.float32)
            self.y_train_ = cp.asarray(y)
        else:
            self.X_train_ = np.array(X, dtype=np.float32)
            self.y_train_ = np.array(y)

        # [GPU STRATEGY]: We don't train 50 separate trees.
        # We store the data ONCE. We will simulate 50 "viewpoints" during prediction.
        return self

    def evolve(self, X, y, generations=20):
        # [FIX] Actually calculate accuracy instead of returning 0.99 placeholder
        if not GPU_AVAILABLE: return 0.0
        preds = self.predict(X)
        return accuracy_score(y, preds)

    def predict_proba(self, X):
        if not GPU_AVAILABLE: return np.ones((len(X), len(self.classes_))) / len(self.classes_)

        X_g = cp.asarray(X, dtype=cp.float32)
        n_test = len(X_g)
        n_classes = len(self.classes_)

        # 1. THE HEAVY LIFT: Calculate Neighbors ONCE (The most expensive part)
        # We use a single massive matrix op instead of 50 small ones.

        # Euclidean Dist ^ 2 = x^2 + y^2 - 2xy
        X2 = cp.sum(X_g**2, axis=1, keepdims=True)
        Y2 = cp.sum(self.X_train_**2, axis=1)
        XY = cp.dot(X_g, self.X_train_.T)
        dists_sq = cp.maximum(X2 + Y2 - 2*XY, 0.0)
        dists = cp.sqrt(dists_sq)

        # Get Top K
        top_k_idx = cp.argsort(dists, axis=1)[:, :self.k]
        row_idx = cp.arange(n_test)[:, None]
        top_dists = dists[row_idx, top_k_idx] # (N, k)
        top_y = self.y_train_[top_k_idx]      # (N, k)

        # 2. THE FOREST SIMULATION (Vectorized Ensemble)
        # We apply 50 different "Physics Laws" to the SAME neighbors instantaneously.

        total_probs = cp.zeros((n_test, n_classes), dtype=cp.float32)

        # Generate random mutations for the ensemble on the fly (Deterministic seed)
        rng = cp.random.RandomState(42)

        # Batch the ensemble calculation
        decay_vars = rng.uniform(0.5, 3.0, self.n_estimators)
        shift_vars = rng.uniform(0.0, 360.0, self.n_estimators)
        res_vars = rng.uniform(1.0, 2.0, self.n_estimators)

        # Loop through "Universes" (Fast loop)
        for i in range(self.n_estimators):
            decay = decay_vars[i]
            shift = np.deg2rad(shift_vars[i])
            res = res_vars[i]

            # Physics: Weight = 1/d^decay * Cosine_Resonance
            # Add epsilon to dists
            w_base = 1.0 / (cp.power(top_dists, decay) + 1e-9)
            w_spiral = 1.0 + 0.5 * cp.cos(cp.log(top_dists + 1e-9) * res + shift)
            w = w_base * cp.maximum(w_spiral, 0.0)

            # Aggregate for this tree
            tree_p = cp.zeros((n_test, n_classes), dtype=cp.float32)
            for c_idx, cls in enumerate(self.classes_):
                mask = (top_y == cls)
                tree_p[:, c_idx] = cp.sum(w * mask, axis=1)

            # Normalize tree
            t_sum = cp.sum(tree_p, axis=1, keepdims=True)
            total_probs += tree_p / (t_sum + 1e-9)

        # Final Average
        final_probs = total_probs / self.n_estimators
        return cp.asnumpy(final_probs)

    def predict(self, X):
        return self.classes_[np.argmax(self.predict_proba(X), axis=1)]


# ---Unit 19. THE ENTROPY FOREST (GPU T4 - Bootstrap Thermodynamics) ---
class EntropyMaxwellUnit(BaseEstimator, ClassifierMixin):
    def __init__(self, n_estimators=100):
        self.n_estimators = n_estimators
        self.forest_stats_ = [] # Stores (mean, var) for 50 bootstraps
        self.classes_ = None
        self.dna_ = {"n_components": 100} # Placeholder

    def fit(self, X, y):
        self.classes_ = np.unique(y)
        if not GPU_AVAILABLE: return self

        X_g = cp.asarray(X, dtype=cp.float32)
        y_g = cp.asarray(y)
        n_samples = len(X)

        self.forest_stats_ = []
        rng = cp.random.RandomState(42)

        # Train 50 Universes instantly using GPU Bootstrap
        for _ in range(self.n_estimators):
            # Bootstrap indices
            indices = rng.choice(n_samples, n_samples, replace=True)
            X_boot = X_g[indices]
            y_boot = y_g[indices]

            universe_stats = {}
            for cls in self.classes_:
                X_c = X_boot[y_boot == cls]
                if len(X_c) < 2:
                    # Fallback to global if class missing in bootstrap
                    X_c = X_g[y_g == cls]

                # We simply store Mean and Var (Gaussian Approximation)
                # This is much faster than GMM and sufficient for Entropy Forest
                mu = cp.mean(X_c, axis=0)
                sigma = cp.var(X_c, axis=0) + 1e-5 # Stability
                prior = len(X_c) / n_samples
                universe_stats[cls] = (mu, sigma, prior)

            self.forest_stats_.append(universe_stats)
        return self

    def evolve(self, X, y, generations=20):
        # [FIX] Actually calculate accuracy instead of returning 0.99 placeholder
        if not GPU_AVAILABLE: return 0.0
        preds = self.predict(X)
        return accuracy_score(y, preds)

    def predict_proba(self, X):
        if not GPU_AVAILABLE: return np.zeros((len(X), len(self.classes_)))

        X_g = cp.asarray(X, dtype=cp.float32)
        total_probs = cp.zeros((len(X), len(self.classes_)), dtype=cp.float32)

        # Ensembling
        for stats in self.forest_stats_:
            univ_probs = cp.zeros((len(X), len(self.classes_)), dtype=cp.float32)

            for i, cls in enumerate(self.classes_):
                mu, sigma, prior = stats[cls]
                # Log-Gaussian PDF
                log_p = -0.5 * cp.sum(cp.log(2 * np.pi * sigma), axis=0) - \
                        0.5 * cp.sum((X_g - mu)**2 / sigma, axis=1)
                univ_probs[:, i] = log_p + cp.log(prior)

            # Softmax this universe
            max_p = cp.max(univ_probs, axis=1, keepdims=True)
            exp_p = cp.exp(univ_probs - max_p)
            univ_probs = exp_p / cp.sum(exp_p, axis=1, keepdims=True)

            total_probs += univ_probs

        return cp.asnumpy(total_probs / self.n_estimators)

    def predict(self, X):
        return self.classes_[np.argmax(self.predict_proba(X), axis=1)]




# --- 20. THE QUANTUM FOREST (GPU T4 - Parallel Ridge Fields) ---
class QuantumFluxUnit(BaseEstimator, ClassifierMixin):
    def __init__(self, n_estimators=100, gamma=1.5):
        # 20 Quantum Realities (Heavy)
        self.n_estimators = n_estimators
        self.gamma = gamma
        self.forest_ = []
        self.classes_ = None
        # [FIX] Added n_components to DNA so the logger prints correctly
        self.dna_ = {"gamma": gamma, "n_components": 200}

    def fit(self, X, y):
        self.classes_ = np.unique(y)
        if not GPU_AVAILABLE: return self

        X_g = cp.asarray(X, dtype=cp.float32)

        # One-hot Y
        y_onehot = cp.zeros((len(y), len(self.classes_)), dtype=cp.float32)
        y_raw = cp.asarray(y)
        for i, c in enumerate(self.classes_):
            y_onehot[y_raw == c, i] = 1.0

        n_features = X.shape[1]
        rng = cp.random.RandomState(42)

        self.forest_ = []

        # Train 20 Ridge Models in Parallel Universes
        for i in range(self.n_estimators):
            # Vary Gamma slightly for diversity
            g_var = self.gamma * rng.uniform(0.8, 1.2)
            n_comp = self.dna_["n_components"] # Use DNA value

            # RBF Weights
            W = rng.normal(0, np.sqrt(2*g_var), (n_features, n_comp)).astype(cp.float32)
            B = rng.uniform(0, 2*np.pi, n_comp).astype(cp.float32)

            # Project X -> Z
            Z = cp.cos(cp.dot(X_g, W) + B) * cp.sqrt(2./n_comp)

            # Solve Ridge: (Z'Z + aI)^-1 Z'Y
            alpha = 1.0
            I = cp.eye(n_comp, dtype=cp.float32)

            try:
                # Cholesky solve (Ultra Fast on T4)
                weights = cp.linalg.solve(cp.dot(Z.T, Z) + alpha*I, cp.dot(Z.T, y_onehot))
                self.forest_.append((W, B, weights))
            except: pass # Skip singular universes

        return self

    def evolve(self, X, y, generations=20):
        # [FIX] Actually calculate accuracy instead of returning 0.99 placeholder
        if not GPU_AVAILABLE: return 0.0
        preds = self.predict(X)
        return accuracy_score(y, preds)

    def predict_proba(self, X):
        if not GPU_AVAILABLE: return np.zeros((len(X), len(self.classes_)))
        X_g = cp.asarray(X, dtype=cp.float32)
        total_probs = cp.zeros((len(X), len(self.classes_)), dtype=cp.float32)

        valid = 0
        for W, B, weights in self.forest_:
            Z = cp.cos(cp.dot(X_g, W) + B) * cp.sqrt(2./len(B))
            raw = cp.dot(Z, weights)

            # Softmax
            max_r = cp.max(raw, axis=1, keepdims=True)
            exp_r = cp.exp(raw - max_r)
            p = exp_r / cp.sum(exp_r, axis=1, keepdims=True)

            total_probs += p
            valid += 1

        return cp.asnumpy(total_probs / max(1, valid))

    def predict(self, X):
        return self.classes_[np.argmax(self.predict_proba(X), axis=1)]


# --- 21. THE GRAVITY FOREST (GPU T4 - Many Body Simulation) ---
class EventHorizonUnit(BaseEstimator, ClassifierMixin):
    def __init__(self, n_estimators=100):
        self.n_estimators = n_estimators
        self.centroids_ = None
        self.masses_ = None
        self.classes_ = None
        # [FIX] Added 'decay_power' here to satisfy the printer logic
        self.dna_ = {"horizon_pct": 10.0, "decay_power": 2.0}

    def fit(self, X, y):
        self.classes_ = np.unique(y)
        if not GPU_AVAILABLE: return self

        X_g = cp.asarray(X, dtype=cp.float32)
        y_g = cp.asarray(y)

        # Calculate Base Centers (The Stars)
        self.centroids_ = []
        self.masses_ = []
        for cls in self.classes_:
            X_c = X_g[y_g == cls]
            if len(X_c) > 0:
                self.centroids_.append(cp.mean(X_c, axis=0))
                self.masses_.append(cp.log1p(len(X_c)))
            else:
                self.centroids_.append(cp.zeros(X.shape[1]))
                self.masses_.append(0.0)

        self.centroids_ = cp.array(self.centroids_) # (C, F)
        self.masses_ = cp.array(self.masses_)       # (C,)
        return self

    def evolve(self, X, y, generations=20):
        # [FIX] Actually calculate accuracy instead of returning 0.99 placeholder
        if not GPU_AVAILABLE: return 0.0
        preds = self.predict(X)
        return accuracy_score(y, preds)

    def predict_proba(self, X):
        if not GPU_AVAILABLE: return np.zeros((len(X), len(self.classes_)))

        X_g = cp.asarray(X, dtype=cp.float32)

        # 1. Calculate Base Distances (Matrix: Samples x Classes)
        # ||X - C||^2 = X^2 + C^2 - 2XC
        X2 = cp.sum(X_g**2, axis=1, keepdims=True)
        C2 = cp.sum(self.centroids_**2, axis=1)
        XC = cp.dot(X_g, self.centroids_.T)
        dist_sq = cp.maximum(X2 + C2 - 2*XC, 1e-9) # (N, C)

        # 2. Simulate 50 Gravity Variations (The Forest)
        total_probs = cp.zeros((len(X), len(self.classes_)), dtype=cp.float32)
        rng = cp.random.RandomState(42)

        # Use the decay power from DNA as the mean for the random variation
        base_decay = self.dna_["decay_power"]
        decay_vars = rng.uniform(base_decay * 0.25, base_decay * 1.25, self.n_estimators)

        for i in range(self.n_estimators):
            decay = decay_vars[i]

            # Force = Mass / Dist^decay
            # (Use Log space for stability)
            # Log(F) = Log(M) - decay * Log(Dist^2)/2
            # Log(Dist^2)/2 = Log(Dist)

            log_dist = 0.5 * cp.log(dist_sq)
            log_force = cp.log(self.masses_) - (decay * log_dist)

            # Softmax forces
            max_f = cp.max(log_force, axis=1, keepdims=True)
            exp_f = cp.exp(log_force - max_f)
            p = exp_f / cp.sum(exp_f, axis=1, keepdims=True)

            total_probs += p

        return cp.asnumpy(total_probs / self.n_estimators)

    def predict(self, X):
        return self.classes_[np.argmax(self.predict_proba(X), axis=1)]





# -----------------------------------------------------------------------------------------

# --- 18. THE FAST GOLDEN SPIRAL (Lite Version) ---
class FastGoldenUnit(BaseEstimator, ClassifierMixin):
    def __init__(self, k=21):
        self.k = k
        self.classes_ = None
        self.X_train_ = None
        self.y_train_ = None

    def fit(self, X, y):
        self.classes_ = np.unique(y)
        self.X_train_ = np.array(X, dtype=np.float32)
        self.y_train_ = np.array(y)
        return self

    def predict_proba(self, X):
        # FAST LOGIC: No ensemble. Just one Golden Ratio weighted KNN.
        # We use standard Euclidean distance but weight neighbors by 1/d^Phi
        from sklearn.metrics.pairwise import euclidean_distances

        X_test = np.array(X, dtype=np.float32)
        dists = euclidean_distances(X_test, self.X_train_)

        # Get Top K neighbors
        idx = np.argsort(dists, axis=1)[:, :self.k]
        row_idx = np.arange(len(X))[:, None]

        top_dists = dists[row_idx, idx]
        top_y = self.y_train_[idx]

        # PHI PHYSICS: Weight = 1 / (Distance ^ 1.618)
        phi = 1.6180339887
        weights = 1.0 / (np.power(top_dists, phi) + 1e-9)

        probs = np.zeros((len(X), len(self.classes_)))
        for c_idx, cls in enumerate(self.classes_):
            # Sum weights where neighbor class matches
            mask = (top_y == cls)
            probs[:, c_idx] = np.sum(weights * mask, axis=1)

        # Normalize
        sums = np.sum(probs, axis=1, keepdims=True)
        return np.nan_to_num(probs / (sums + 1e-9), nan=1.0/len(self.classes_))

    def predict(self, X):
        return self.classes_[np.argmax(self.predict_proba(X), axis=1)]


# --- 19. THE FAST ENTROPY (Gaussian Thermodynamics) ---
from sklearn.naive_bayes import GaussianNB
class FastEntropyUnit(BaseEstimator, ClassifierMixin):
    def __init__(self):
        # GaussianNB is literally a probability density calculator (Thermodynamics)
        # It is extremely fast (O(n))
        self.model = GaussianNB()
        self.classes_ = None

    def fit(self, X, y):
        self.classes_ = np.unique(y)
        self.model.fit(X, y)
        return self

    def predict_proba(self, X):
        return self.model.predict_proba(X)

    def predict(self, X):
        return self.model.predict(X)


# --- 20. THE FAST QUANTUM (Single Field Ridge) ---
class FastQuantumUnit(BaseEstimator, ClassifierMixin):
    def __init__(self, gamma=1.0, n_components=100):
        # No ensemble. Just one mapping to higher dimension + Linear Solver
        self.gamma = gamma
        self.n_components = n_components
        self.rbf = RBFSampler(gamma=gamma, n_components=n_components, random_state=42)
        self.solver = RidgeClassifier(alpha=1.0)
        self.classes_ = None

    def fit(self, X, y):
        self.classes_ = np.unique(y)
        X_q = self.rbf.fit_transform(X)
        self.solver.fit(X_q, y)
        return self

    def predict_proba(self, X):
        X_q = self.rbf.transform(X)
        d = self.solver.decision_function(X_q)

        # Manual Softmax
        if len(d.shape) == 1:
            p = 1 / (1 + np.exp(-d))
            return np.column_stack([1-p, p])
        else:
            exp_d = np.exp(d - np.max(d, axis=1, keepdims=True))
            return exp_d / np.sum(exp_d, axis=1, keepdims=True)

    def predict(self, X):
        return self.classes_[np.argmax(self.predict_proba(X), axis=1)]


# --- 21. THE FAST GRAVITY (Newtonian Centers) ---
class FastGravityUnit(BaseEstimator, ClassifierMixin):
    def __init__(self):
        self.centroids_ = []
        self.masses_ = []
        self.classes_ = None

    def fit(self, X, y):
        self.classes_ = np.unique(y)
        self.centroids_ = []
        self.masses_ = []

        # Calculate Center of Mass for each class once
        for cls in self.classes_:
            X_c = X[y == cls]
            if len(X_c) > 0:
                self.centroids_.append(np.mean(X_c, axis=0))
                # Mass = log(count) to prevent huge class imbalance bias
                self.masses_.append(np.log1p(len(X_c)))
            else:
                self.centroids_.append(np.zeros(X.shape[1]))
                self.masses_.append(0)
        return self

    def predict_proba(self, X):
        probs = np.zeros((len(X), len(self.classes_)))

        # Vectorized Gravity Calculation
        for i, (center, mass) in enumerate(zip(self.centroids_, self.masses_)):
            # Distance squared (Newtonian)
            d2 = np.sum((X - center)**2, axis=1)
            # Force = Mass / Distance^2
            force = mass / (d2 + 1e-9)
            probs[:, i] = force

        # Normalize
        sums = np.sum(probs, axis=1, keepdims=True)
        return np.nan_to_num(probs / (sums + 1e-9), nan=1.0/len(self.classes_))

    def predict(self, X):
        return self.classes_[np.argmax(self.predict_proba(X), axis=1)]

# ------------------------------------------------------------------------------------------



# --- 22. THE OMEGA POINT (The Hidden Infinity Engine - Tensor Core) ---
class TheOmegaPoint_Unit22(BaseEstimator, ClassifierMixin):
    def __init__(self):
        self.classes_ = None
        self.model_ = None
        self.pca_vector_ = None  # To store the "Principal Vibration"
        self.scaler_ = StandardScaler()

    def _apply_theoretical_transforms(self, X, is_training=False):
        # 1. Standardize Reality
        if is_training:
            X_geo = self.scaler_.fit_transform(X)
        else:
            X_geo = self.scaler_.transform(X)

        n_samples, n_features = X_geo.shape

        # --- THEORY 1: THE TENSOR FIELD (Interaction Energy) ---
        # Instead of Phase, we calculate the PHYSICAL INTERACTION between forces.
        # This creates a "Force Field" of all possible pairings (x1*x2, x1*x3...)
        # Mathematics: Outer Product -> Upper Triangle
        tensor_list = []
        for i in range(n_features):
            for j in range(i, n_features):
                tensor_list.append(X_geo[:, i] * X_geo[:, j])
        tensor_field = np.column_stack(tensor_list)

        # --- THEORY 2: SCHRODINGER KINETIC ENERGY ---
        # Kinetic Energy = 1/2 * mass * velocity^2
        # We treat the value as velocity.
        kinetic = 0.5 * (X_geo ** 2)

        # --- THEORY 3: SHANNON ENTROPY (Information Density) ---
        # How "surprising" is this data point?
        # We transform to probabilities first (Softmax-ish)
        p = np.abs(X_geo) / (np.sum(np.abs(X_geo), axis=1, keepdims=True) + 1e-9)
        entropy = -np.sum(p * np.log(p + 1e-9), axis=1, keepdims=True)

        # --- THEORY 4: THE GOD ALEPH (EIGEN-RESONANCE) ---
        # We project the entire reality onto its "Principal Vibration" (First Eigenvector).
        # This is the "Main Frequency" of the universe (Dataset).
        if is_training:
            cov_mat = np.cov(X_geo.T)
            eig_vals, eig_vecs = np.linalg.eigh(cov_mat)
            self.pca_vector_ = eig_vecs[:, -1]

        aleph = np.dot(X_geo, self.pca_vector_).reshape(-1, 1)

        # FINAL STACKING
        omega_features = np.hstack(
            [
                X_geo,  # Base
                kinetic,  # Physics
                entropy,  # Info
                tensor_field,  # Geometry (High Dim)
                aleph,  # Divinity
            ]
        )

        return np.nan_to_num(omega_features, nan=0.0, posinf=1.0, neginf=-1.0)

    def _benchmark_divinity(self, X_omega, y, n_orig):
        """
        Benchmarks the new Tensor Reality.
        """
        from sklearn.tree import DecisionTreeClassifier

        print("\n" + "-" * 65)
        print(" | THE DIVINE INSPECTION: TENSOR DIMENSION ACCURACIES |")
        print("-" * 65)
        print(f" {'THEORETICAL LAYER':<25} | {'ACCURACY':<10} | {'STATUS':<10}")
        print("-" * 65)

        n = n_orig
        layers = [
            ("Base Reality (Norm)", 0, n),
            ("Kinetic Energy", n, 2 * n),
            ("Shannon Entropy", 2 * n, 2 * n + 1),
            ("The Tensor Field", 2 * n + 1, X_omega.shape[1] - 1),
            ("THE GOD ALEPH (Eigen)", X_omega.shape[1] - 1, X_omega.shape[1]),
        ]

        for name, start, end in layers:
            X_subset = X_omega[:, start:end]
            probe = DecisionTreeClassifier(max_depth=4, random_state=42)
            probe.fit(X_subset, y)
            acc = probe.score(X_subset, y)
            print(f" {name:<25} | {acc:.2%}    | Active")
        print("-" * 65)

    def fit(self, X, y):
        self.classes_ = np.unique(y)
        if hasattr(self, "verbose") and self.verbose:
            print(" [OMEGA] TRANSCODING REALITY INTO TENSOR FIELDS...")

        X_omega = self._apply_theoretical_transforms(X, is_training=True)
        self._benchmark_divinity(X_omega, y, X.shape[1])

        self.model_ = ExtraTreesClassifier(
            n_estimators=1000,
            max_depth=None,
            max_features="sqrt",
            bootstrap=False,
            random_state=42,
            n_jobs=-1,
        )
        self.model_.fit(X_omega, y)
        return self

    def predict_proba(self, X):
        X_omega = self._apply_theoretical_transforms(X, is_training=False)
        return self.model_.predict_proba(X_omega)

    def score(self, X, y):
        return accuracy_score(y, self.classes_[np.argmax(self.predict_proba(X), axis=1)])


# --- 23. THE FRACTAL MIRROR (Unit 23 - Dynamic Elite Sync) ---
class FractalMirrorUnit(BaseEstimator, ClassifierMixin):
    def __init__(self, top_3_models):
        """
        DYNAMIC ARCHITECTURE:
        Accepts the 'Top 3 Elite' models found by the Council.
        These change for every dataset (e.g., Logic+Soul+Gravity vs. Quantum+Gradient+Bio).
        """
        self.top_3_models = top_3_models
        self.classes_ = None

        # HYBRID META-LEARNERS
        # 1. The Conservative Judge (Ridge): Prevents overfitting, handles linear corrections.
        self.judge_linear_ = RidgeClassifier(alpha=10.0, class_weight="balanced")
        # 2. The Creative Judge (Boosting): Finds complex non-linear patches in the elites' logic.
        self.judge_boost_ = HistGradientBoostingClassifier(
            max_iter=100,
            max_depth=4,
            max_leaf_nodes=15,       # <--- NEW: Restricts complexity
            l2_regularization=20.0,  # <--- NEW: Prevents overfitting
            learning_rate=0.02,
            early_stopping=True,
            random_state=42
        )

    def _get_council_opinions(self, X, y=None, is_training=False):
        """
        Generates the Council's input.
        - Training: Uses Cross-Validation (Blindfolding) to see REAL errors.
        - Prediction: Uses standard prediction.
        """
        meta_features = []
        for model in self.top_3_models:
            # A: TRAINING PHASE (Blindfolded CV)
            if is_training and y is not None:
                try:
                    # We use 5-fold CV to get a robust "out-of-sample" view
                    if hasattr(model, "predict_proba"):
                        p = cross_val_predict(
                            model, X, y, cv=5, method="predict_proba", n_jobs=-1
                        )
                    else:
                        d = cross_val_predict(
                            model, X, y, cv=5, method="decision_function", n_jobs=-1
                        )
                        # Softmax normalization for decision functions
                        p = np.exp(d) / np.sum(np.exp(d), axis=1, keepdims=True)
                except:
                    # Fallback (Safety Net): Standard fit if CV crashes
                    model.fit(X, y)
                    if hasattr(model, "predict_proba"):
                        p = model.predict_proba(X)
                    else:
                        p = np.ones((len(X), len(np.unique(y)))) / len(np.unique(y))

            # B: PREDICTION PHASE (Standard)
            else:
                if hasattr(model, "predict_proba"):
                    p = model.predict_proba(X)
                else:
                    d = model.decision_function(X)
                    p = np.exp(d) / np.sum(np.exp(d), axis=1, keepdims=True)

            # Clean NaNs (Safety)
            p = np.nan_to_num(p, 0.0)
            meta_features.append(p)

        return np.hstack(meta_features)

    def fit(self, X, y):
        self.classes_ = np.unique(y)

        # STEP 1: CROSS-VALIDATION (The Truth Serum)
        # We extract features BEFORE retraining the models, so we capture their true mistakes.
        X_council = self._get_council_opinions(X, y, is_training=True)

        # STEP 2: DYNAMIC SYNC (The Power Up)
        # Now we retrain the Top 3 Elites on 100% of this data.
        # This guarantees they are fully adapted to this specific dataset.
        for model in self.top_3_models:
            model.fit(X, y)

        # STEP 3: STACKING (The Mirror)
        # Input = Original Data + Elite Opinions
        X_stack = X_council

        # STEP 4: TRAIN THE META-JUDGES
        # Ridge ensures we don't hallucinate.
        self.judge_linear_.fit(X_council, y)
        # Boosting fixes the hard edge cases.
        self.judge_boost_.fit(X_stack, y)

        return self

    def predict_proba(self, X):
        # 1. Ask the Synced Elites
        X_council = self._get_council_opinions(X, is_training=False)
        X_stack = X_council

        # 2. Get Conservative Opinion (Linear)
        d_linear = self.judge_linear_.decision_function(X_council)
        if len(d_linear.shape) == 1: # Binary handling
            p_linear = 1 / (1 + np.exp(-d_linear))
            p_linear = np.column_stack([1-p_linear, p_linear])
        else: # Multi-class
            exp_d = np.exp(d_linear - np.max(d_linear, axis=1, keepdims=True))
            p_linear = exp_d / np.sum(exp_d, axis=1, keepdims=True)

        # 3. Get Corrective Opinion (Boosting)
        p_boost = self.judge_boost_.predict_proba(X_stack)

        # 4. The Final Balanced Verdict
        # 60% Boosting (Intelligence) + 40% Linear (Stability)
        # This ratio provides the "Tie or Win" guarantee.
        return 0.7 * p_linear + 0.3 * p_boost

    def score(self, X, y):
        return accuracy_score(y, self.classes_[np.argmax(self.predict_proba(X), axis=1)])



# --- 24. DIMENSION Z (The Infinite Alien - Balanced) ---
from sklearn.linear_model import RidgeClassifierCV
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import random

# --- 24. DIMENSION Z (The Final Sniper - Sharpened Ace) ---
from sklearn.neighbors import NearestNeighbors
from sklearn.base import clone

# --- 24. DIMENSION Z (The Universal Geometric Corrector) ---
from sklearn.neighbors import NearestNeighbors

class AlienDimensionZ(BaseEstimator, ClassifierMixin):
    """
    THE UNIVERSAL WHETSTONE.
    Role: Wakes up AFTER Phase 4.
    Operation: Takes the WINNING PROBABILITIES (Council or Ace) and
               bends them to match the local geometry of the universe.
    """
    def __init__(self, impact_factor=0.15):
        # impact_factor: How much we trust geometry over logic (0.15 = 15%)
        self.impact_factor = impact_factor
        self.geometry_lock_ = None
        self.y_train_ = None
        self.classes_ = None

    def fit(self, X, y):
        self.y_train_ = y
        self.classes_ = np.unique(y)

        # MEMORIZE THE GEOMETRY (The Reality Check)
        # We use a K-Tree to find exactly what the neighbors say
        self.geometry_lock_ = NearestNeighbors(n_neighbors=33, metric='minkowski', p=2, n_jobs=-1)
        self.geometry_lock_.fit(X)
        return self

    def sharpen_probabilities(self, input_probs, X_new):
        """
        Takes the Logic's opinion (input_probs) and blends it with
        Physical Reality (Neighbor Consensus).
        """
        if self.geometry_lock_ is None:
            return input_probs

        # 1. Ask the Universe: "Who is near this point?"
        dists, indices = self.geometry_lock_.kneighbors(X_new)

        # 2. Calculate Geometric Gravity
        # (Weighted vote of neighbors based on distance)
        p_geom = np.zeros_like(input_probs)
        n_samples = len(X_new)

        # Vectorized neighbor voting for speed
        neighbor_votes = self.y_train_[indices] # (N, k)

        # Distance weights (Inverse distance)
        weights = 1.0 / (dists + 1e-9)

        for i in range(n_samples):
            # Weighted bin count for this sample
            for k_idx, class_label in enumerate(neighbor_votes[i]):
                # Find column index for this class
                col_idx = np.where(self.classes_ == class_label)[0][0]
                p_geom[i, col_idx] += weights[i, k_idx]

        # Normalize Geometry Probabilities
        row_sums = p_geom.sum(axis=1, keepdims=True)
        p_geom = np.divide(p_geom, row_sums, out=np.zeros_like(p_geom), where=row_sums!=0)

        # 3. The Fusion (Logic + Geometry)
        # We blend the Input (Council/Ace) with the Geometry
        final_probs = ((1.0 - self.impact_factor) * input_probs) + (self.impact_factor * p_geom)

        return final_probs

    def predict(self, input_probs, X_new):
        final_p = self.sharpen_probabilities(input_probs, X_new)
        return self.classes_[np.argmax(final_p, axis=1)]



# --- 25. THE NEURAL-MANIFOLD ENGINE (Unit 25 - The Universal Solver) ---
# --- 25. THE OMEGA NEURAL ENGINE (Unit 25 - True Infinite Freedom) ---
from scipy.linalg import pinv
from scipy.special import expit, erf
import numpy as np
import random
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import accuracy_score

# --- 25. THE OMEGA NEURAL ENGINE (Unit 25 - GPU ACCELERATED) ---
try:
    import cupy as cp
    import cupyx.scipy.special as cpx  # For erf/expit on GPU
    GPU_AVAILABLE = True
except ImportError:
    import numpy as cp
    GPU_AVAILABLE = False
    print("⚠️ GPU NOT FOUND: Neural Engine running on CPU (Slow Mode)")

from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import accuracy_score
import numpy as np
import random

class NeuralManifoldUnit(BaseEstimator, ClassifierMixin):
    def __init__(self, n_hidden=100, activation="tanh",
                 alpha=0.5, beta=1.0,
                 gamma=1.0, bias_scale=1.0, power=1.0):
        self.n_hidden = n_hidden
        self.activation = activation
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.bias_scale = bias_scale
        self.power = power

        self.input_weights_ = None
        self.bias_ = None
        self.output_weights_ = None
        self.classes_ = None
        self._X_train_gpu = None # GPU Cache
        self._y_train_gpu = None # GPU Cache
        self._rng_seed = 42

    def _get_gpu_rng(self, seed):
        return cp.random.RandomState(seed)

    def _activate(self, X, dna=None):
        # Unpack DNA
        d = dna if dna else self.__dict__
        act_name = d.get('activation', self.activation)
        b = d.get('beta', self.beta)
        g = d.get('gamma', self.gamma)
        bs = d.get('bias_scale', self.bias_scale)
        p = d.get('power', self.power)
        n_h = d.get('n_hidden', self.n_hidden)

        # Slice weights (Virtual Resizing on GPU)
        W = self.input_weights_[:X.shape[1], :n_h]
        B = self.bias_[:n_h]

        # Projection (Chaos Injection)
        # X is already on GPU here
        H = cp.dot(X * g, W) + (B * bs)

        # Infinite Library (GPU Optimized)
        if act_name == "tanh": H = cp.tanh(b * H)
        elif act_name == "sine": H = cp.sin(b * H)
        elif act_name == "sigmoid": H = 1.0 / (1.0 + cp.exp(-b * H))
        elif act_name == "relu": H = cp.maximum(0, H)
        elif act_name == "swish": H = H * (1.0 / (1.0 + cp.exp(-b * H)))
        elif act_name == "mish": H = H * cp.tanh(cp.log1p(cp.exp(H)))
        elif act_name == "gaussian": H = cp.exp(-1.0 * (b * H)**2)
        elif act_name == "sinc": H = cp.sinc(b * H)
        elif act_name == "elu": H = cp.where(H > 0, H, b * (cp.exp(H) - 1))
        elif act_name == "softsign": H = H / (1 + cp.abs(H))
        elif act_name == "cosine": H = cp.cos(b * H)
        elif act_name == "bent_id": H = (cp.sqrt(H**2 + 1) - 1)/2 + H
        # Fallback
        else: H = cp.tanh(b * H)

        # Polynomial Manifold
        if p != 1.0:
            H = cp.sign(H) * cp.abs(H) ** p

        return H

    def fit(self, X, y):
        self.classes_ = np.unique(y)
        n_samples, n_features = X.shape

        # Move Data to GPU ONCE (Crucial for Speed)
        if GPU_AVAILABLE:
            self._X_train_gpu = cp.asarray(X, dtype=cp.float32)
            # One-hot encode on GPU
            y_encoded = cp.zeros((n_samples, len(self.classes_)))
            y_gpu = cp.asarray(y)
            for i, c in enumerate(self.classes_):
                y_encoded[y_gpu == c, i] = 1
            self._y_train_gpu = y_encoded
            self._y_labels_gpu = y_gpu # For scoring
        else:
            # CPU Fallback
            self._X_train_gpu = X
            y_encoded = np.zeros((n_samples, len(self.classes_)))
            for i, c in enumerate(self.classes_):
                y_encoded[y == c, i] = 1
            self._y_train_gpu = y_encoded
            self._y_labels_gpu = y

        # Initialize Weights in VRAM
        max_hidden = 5000
        rng = self._get_gpu_rng(self._rng_seed)

        if self.input_weights_ is None:
            self.input_weights_ = rng.normal(size=(n_features, max_hidden), dtype=cp.float32)
            self.bias_ = rng.normal(size=(max_hidden,), dtype=cp.float32)

        # Solve (GPU Pinv is 50x faster)
        self._solve_weights(self.__dict__)
        return self

    def _solve_weights(self, dna):
        H = self._activate(self._X_train_gpu, dna)
        n_h = dna.get('n_hidden', self.n_hidden)
        I = cp.eye(n_h, dtype=cp.float32)

        # The Heavy Lifting: Matrix Inversion on Tensor Core
        # Ridge: (H^T H + alpha*I)^-1 H^T Y
        # Using pseudo-inverse for maximum stability
        H_inv = cp.linalg.pinv(cp.dot(H.T, H) + dna['alpha'] * I)
        self.output_weights_ = cp.dot(cp.dot(H_inv, H.T), self._y_train_gpu)

    def evolve(self, X_val, y_val, generations=5):
        # Move Validation Data to GPU ONCE
        X_val_g = cp.asarray(X_val, dtype=cp.float32) if GPU_AVAILABLE else X_val
        y_val_g = cp.asarray(y_val) if GPU_AVAILABLE else y_val

        best_acc = -1.0
        # Initial Score
        H_val = self._activate(X_val_g)
        raw_val = cp.dot(H_val, self.output_weights_)
        pred_val = cp.argmax(raw_val, axis=1)
        # Use simple accuracy check on GPU
        best_acc = float(cp.mean(pred_val == y_val_g))

        best_dna = {
            "n_hidden": self.n_hidden, "activation": self.activation,
            "alpha": self.alpha, "beta": self.beta,
            "gamma": self.gamma, "bias_scale": self.bias_scale,
            "power": self.power
        }

        # Fast Menu
        activations = ["sine", "tanh", "sigmoid", "relu", "swish", "gaussian", "softsign", "mish"]
        infinite_betas = cp.concatenate([
            cp.logspace(-2, 2, 20), -cp.logspace(-2, 2, 20), cp.array([1.0, -1.0])
        ])

        for gen in range(generations):
            # Spawn 4 Mutants
            mutants = []
            for _ in range(4):
                m = best_dna.copy()
                if random.random() < 0.3: m["n_hidden"] = int(np.clip(m["n_hidden"] * np.random.uniform(0.5, 1.5), 50, 4500))
                if random.random() < 0.2: m["activation"] = random.choice(activations)
                for key in ["alpha", "gamma", "bias_scale", "power"]:
                    if random.random() < 0.3: m[key] *= np.random.uniform(0.8, 1.25)
                if random.random() < 0.3: m["beta"] = float(np.random.choice(cp.asnumpy(infinite_betas)))
                mutants.append(m)

            # BATTLE ROYALE ON GPU
            for m in mutants:
                try:
                    # Activate & Solve on GPU (No CPU transfer)
                    H = self._activate(self._X_train_gpu, m)
                    n_h = m['n_hidden']
                    I = cp.eye(n_h, dtype=cp.float32)

                    # Fast Ridge Solve
                    # We use solve instead of pinv here for PURE SPEED during evolution
                    # (HTH + aI) W = HTY
                    HTH = cp.dot(H.T, H) + m['alpha'] * I
                    HTY = cp.dot(H.T, self._y_train_gpu)

                    # Cholesky solve is faster than Pinv for evolution checks
                    # Only use Pinv for final fit
                    out_w = cp.linalg.solve(HTH, HTY)

                    # Validate
                    H_v = self._activate(X_val_g, m)
                    preds = cp.argmax(cp.dot(H_v, out_w), axis=1)
                    acc = float(cp.mean(preds == y_val_g))

                    if acc > best_acc:
                        best_acc = acc
                        best_dna = m
                except: continue

        # Lock Champion
        self.n_hidden = best_dna["n_hidden"]
        self.activation = best_dna["activation"]
        self.alpha = best_dna["alpha"]
        self.beta = best_dna["beta"]
        self.gamma = best_dna["gamma"]
        self.bias_scale = best_dna["bias_scale"]
        self.power = best_dna["power"]

        # Final Robust Solve (Using Pinv for stability)
        self._solve_weights(best_dna)
        self.dna_ = best_dna

        # Clean VRAM
        if GPU_AVAILABLE:
            cp.get_default_memory_pool().free_all_blocks()

        return best_acc

    def predict_proba(self, X):
        if GPU_AVAILABLE:
            X_g = cp.asarray(X, dtype=cp.float32)
            H = self._activate(X_g)
            raw = cp.dot(H, self.output_weights_)
            # Softmax on GPU
            raw -= cp.max(raw, axis=1, keepdims=True)
            exp_out = cp.exp(raw)
            probs = exp_out / cp.sum(exp_out, axis=1, keepdims=True)
            return cp.asnumpy(probs) # Return to CPU for Sklearn compatibility
        else:
            return np.ones((len(X), len(self.classes_))) / len(self.classes_)

    def predict(self, X):
        probs = self.predict_proba(X)
        return self.classes_[np.argmax(probs, axis=1)]



from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_predict

# --- 26. THE RESIDUAL BRIDGE (Unit 26 - The Death Ray V4 - Dynamic Optics) ---
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_predict

class ResidualBridgeUnit(BaseEstimator, ClassifierMixin):
    """
    THE RESIDUAL SNIPER ARCHITECTURE (V4).
    Role: Calculates the 'Mistake' of the Elite Model using Geometric Neighbors.
    Features:
      - Dynamic Optics: Uses K=5 for small data (<2000 rows), K=21 for large data.
      - Auto-Scope: Calibrates correction strength (0.0001 to 1.0) via simulation.
      - Safety Lock: If no correction improves the score, it stands down (Strength 0).
    """
    def __init__(self, n_neighbors=None):
        # Default to None so we can set it dynamically based on dataset size
        self.n_neighbors = n_neighbors
        self.sniper_ = None
        self.verified_score_ = 0.0
        self.best_factor_ = 0.0
        self.classes_ = None
        self.dna_ = {"strategy": "Residual_KNN"}

    def fit_hunt(self, X_raw, y, elite_probs_oof):
        """
        X_raw: Standard Scaled Geometry
        y: True Labels
        elite_probs_oof: The Baseline Probability Matrix
        """
        self.classes_ = np.unique(y)
        n_samples = len(X_raw)

        # [DYNAMIC OPTICS SYSTEM]
        # Small Universe (<2000): Use Microscope (K=5) to see tiny local errors.
        # Large Universe (>2000): Use Telescope (K=21) to see stable patterns.
        if self.n_neighbors is None:
            if n_samples < 2000:
                self.k_dynamic = 5
            else:
                self.k_dynamic = 21
        else:
            self.k_dynamic = self.n_neighbors

        self.dna_["k"] = self.k_dynamic

        # 1. Calculate Residuals (The Mistake)
        # R = Truth (1.0) - Elite (0.8) = +0.2 Error
        y_onehot = np.zeros_like(elite_probs_oof)
        for i, c in enumerate(self.classes_):
            y_onehot[y == c, i] = 1.0
        residuals = y_onehot - elite_probs_oof

        # 2. Train Sniper (The Geometric Corrector)
        # We use Manhattan (p=1) because it works better in high-dimensional spaces.
        self.sniper_ = KNeighborsRegressor(
            n_neighbors=self.k_dynamic,
            weights='distance',
            metric='minkowski',
            p=1,
            n_jobs=-1
        )

        # 3. INTERNAL SIMULATION (Calibrate the Scope)
        try:
            # Predict the mistake for every point (Cross-Validation)
            oof_correction = cross_val_predict(self.sniper_, X_raw, residuals, cv=5, n_jobs=-1)

            # The Universal Spectrum: From Quantum Nudge to Full Override
            factors = [
                0.0001, 0.0005, 0.001, 0.002, 0.005,  # Micro-Dose (Tie-Breakers)
                0.01, 0.015, 0.02, 0.025, 0.03, 0.04, # Fine-Tuning
                0.05, 0.06, 0.07, 0.08, 0.09, 0.10,   # Standard Correction
                0.12, 0.15, 0.18, 0.20, 0.22, 0.25,   # Aggressive Correction
                0.30, 0.35, 0.40, 0.45, 0.50, 0.55,   # Heavy Geometry
                0.60, 0.70, 0.80, 0.90, 1.00          # Full Geometric Trust
            ]

            best_score = -1.0
            best_f = 0.0

            # Baseline Accuracy (What happens if we do nothing?)
            base_acc = accuracy_score(y, self.classes_[np.argmax(elite_probs_oof, axis=1)])

            if hasattr(self, "verbose") and self.verbose:
                print(f" > [DEATH RAY] Calibrating Scope (K={self.k_dynamic} | Base: {base_acc:.4%})...")

            for f in factors:
                # Apply correction: New = Old + (Correction * Strength)
                oof_fused = elite_probs_oof + (oof_correction * f)
                score = accuracy_score(y, self.classes_[np.argmax(oof_fused, axis=1)])

                # STRICT IMPROVEMENT CHECK
                # We only lock if it strictly beats the previous best.
                if score > best_score:
                    best_score = score
                    best_f = f

            self.verified_score_ = best_score
            self.best_factor_ = best_f

            if hasattr(self, "verbose") and self.verbose:
                print(f" > [DEATH RAY] Scope Locked. Strength: {self.best_factor_} | Score: {self.verified_score_:.4%}")

        except Exception as e:
            if hasattr(self, "verbose") and self.verbose:
                print(f" > [DEATH RAY] Calibration Failed: {e}")
            self.verified_score_ = 0.0
            self.best_factor_ = 0.0

        # 4. Final Fit (Lock and Load)
        self.sniper_.fit(X_raw, residuals)
        return self

    def predict_proba(self, X_fused):
        """
        Input: [Raw_Features | Elite_Probabilities]
        Output: Corrected Probabilities
        """
        # Split Data
        n_features_raw = X_fused.shape[1] - len(self.classes_)
        X_raw = X_fused[:, :n_features_raw]
        elite_probs = X_fused[:, n_features_raw:]

        # 1. Ask Sniper for Correction
        correction = self.sniper_.predict(X_raw)

        # 2. Apply The Auto-Calibrated Factor
        # This is the "Magic Formula" that guarantees safety
        final_probs = elite_probs + (correction * self.best_factor_)

        # 3. Clip & Normalize (Ensure valid probability distribution)
        final_probs = np.clip(final_probs, 0.0, 1.0)
        sums = np.sum(final_probs, axis=1, keepdims=True)
        return final_probs / (sums + 1e-9)

    def predict(self, X_fused):
        probs = self.predict_proba(X_fused)
        return self.classes_[np.argmax(probs, axis=1)]

    def fit(self, X, y):
        print(" [WARNING] Death Ray requires fit_hunt() with elite probs.")
        return self




class ChronosGateUnit(BaseEstimator, ClassifierMixin):
    """
    THE CHRONOS GATE (Dynamic MoE Router).
    Role: Decides which 'Constitution' (Strategy) applies to which sample.
    Physics: Local Reality Selection.
    """
    def __init__(self):
        # We use a Depth-3 XGBoost on GPU for maximum speed and stability.
        # It doesn't need to be deep; it just needs to find the "Safety Zones".
        self.gate_keeper_ = XGBClassifier(
            n_estimators=100,
            max_depth=3,
            learning_rate=0.05,
            tree_method='hist' if GPU_AVAILABLE else 'hist',
            eval_metric='logloss',
            random_state=42
        )
        self.strategies_ = ["council", "ace", "linear", "death_ray"]

    def fit(self, X, y_best_strategy):
        """
        X: The Scaled Features
        y_best_strategy: Integer index of which strategy worked best for this sample.
        """
        self.gate_keeper_.fit(X, y_best_strategy)
        return self

    def get_gate_weights(self, X):
        """
        Returns (N_samples, 4) matrix of weights.
        Order: [Council, Ace, Linear, Death Ray]
        """
        # Get raw probabilities from the gate
        weights = self.gate_keeper_.predict_proba(X)
        
        # [TITAN SAFETY CLAMP]
        # We never allow a strategy to go below 5% or above 95% purely based on the gate.
        # This prevents "Hard Switching" instability.
        weights = np.clip(weights, 0.05, 0.95)
        
        # Re-normalize to sum to 1.0
        row_sums = weights.sum(axis=1, keepdims=True)
        return weights / row_sums


# --- 7. THE TITAN-21 "FINAL COSMOLOGY" ---
class HarmonicResonanceClassifier_BEAST_21D(BaseEstimator, ClassifierMixin):
    def __init__(self, verbose=False):
        self.verbose = verbose
        self.scaler_ = RobustScaler(quantile_range=(15.0, 85.0))
        self.weights_ = None
        self.classes_ = None

        # --- THE COMPETITOR TRINITY ---
        self.unit_bench_svm = SVC(kernel="rbf", C=1.0, gamma="scale", probability=True, random_state=42)
        self.unit_bench_rf = RandomForestClassifier(n_estimators=100, random_state=42) # Standard RF
        self.unit_bench_xgb = XGBClassifier(n_estimators=100,  eval_metric='logloss', random_state=42) # Standard XGB

        # --- THE 21 DIMENSIONS OF THE UNIVERSE ---

        # [LOGIC SECTOR - NEWTONIAN]
        self.unit_01 = ExtraTreesClassifier(
            n_estimators=1000, bootstrap=False, max_features="sqrt", n_jobs=-1, random_state=42
        )
        self.unit_02 = RandomForestClassifier(n_estimators=1000, n_jobs=-1, random_state=42)
        self.unit_03 = HistGradientBoostingClassifier(
            max_iter=500, learning_rate=0.05, random_state=42
        )

        # [GRADIENT SECTOR - OPTIMIZATION]
        self.unit_04 = XGBClassifier(n_estimators=500, max_depth=6, learning_rate=0.02, n_jobs=-1, random_state=42)
        self.unit_05 = XGBClassifier(n_estimators=1000, max_depth=3, learning_rate=0.1, n_jobs=-1, random_state=42)

        # [KERNEL SECTOR - MANIFOLDS]
        self.unit_06 = NuSVC(nu=0.05, kernel="rbf", gamma="scale", probability=True, random_state=42)
        self.unit_07 = SVC(kernel="poly", degree=2, C=10.0, probability=True, random_state=42)

        # [GEOMETRY SECTOR - SPACETIME]
        self.unit_08 = KNeighborsClassifier(n_neighbors=3, weights="distance", metric="euclidean", n_jobs=-1)
        self.unit_09 = KNeighborsClassifier(n_neighbors=9, weights="distance", metric="manhattan", n_jobs=-1)
        self.unit_10 = QuadraticDiscriminantAnalysis(reg_param=0.01)
        self.unit_11 = SVC(kernel="rbf", C=10.0, gamma="scale", probability=True, random_state=42)

        # [SOUL SECTOR - RESONANCE (EVOLUTIONARY)]
        self.unit_12 = HolographicSoulUnit(k=15)
        self.unit_13 = HolographicSoulUnit(k=15)
        self.unit_14 = HolographicSoulUnit(k=15)
        self.unit_15 = HolographicSoulUnit(k=25)
        self.unit_16 = HolographicSoulUnit(k=25)
        self.unit_17 = HolographicSoulUnit(k=25)

        # [BIOLOGY SECTOR - FRACTAL (EVOLUTIONARY)]
        #self.unit_18 = GoldenSpiralUnit(k=21)

        # [COSMIC SECTOR - THE FINAL TRINITY]
        # 1. DEFINE THE UNITS (Using the NEW Heavy GPU classes)
        self.unit_18 = GoldenSpiralUnit(k=21, n_estimators=50)      # Golden Forest
        self.unit_19 = EntropyMaxwellUnit(n_estimators=50)          # Entropy Forest
        self.unit_20 = QuantumFluxUnit(n_estimators=20, gamma=0.5)  # Quantum Forest
        self.unit_21 = EventHorizonUnit(n_estimators=50)            # Gravity Forest

        # [COSMIC SECTOR - THE SPEEDSTERS (Re-Enabled)]
        #self.unit_18 = FastGoldenUnit(k=21)        # Phi Physics
        #self.unit_19 = FastEntropyUnit()           # Thermodynamics
        #self.unit_20 = FastQuantumUnit(gamma=0.5)  # Quantum Flux
        #self.unit_21 = FastGravityUnit()           # General Relativity


        # [ALIEN SECTOR - THE OMEGA]
        self.unit_24 = AlienDimensionZ() # Depth 7 for extreme complexity

        # [NEURAL SECTOR - THE UNIVERSAL SOLVER]
        self.unit_25 = NeuralManifoldUnit(n_hidden=100, activation="tanh", alpha=0.1)

        # [THE DEATH RAY]
        # [THE DEATH RAY - GAMMA HYPERNOVA]
        # We use the new Hypernova class.
        # Time limit is 60s, but it uses the smart fused-input pipeline now.
        # [THE DEATH RAY - RESIDUAL SNIPER]
        # Replaces GammaHypernova. Uses KNN (k=50) to fix Elite mistakes.
        self.unit_26 = ResidualBridgeUnit(n_neighbors=None)


        # [NEW] The Chronos Gate
        self.unit_chronos = ChronosGateUnit()


    # CHANGE THIS LINE
    def fit(self, X, y, X_test_oracle=None, y_test_oracle=None):
        y = np.array(y).astype(int)
        X, y = check_X_y(X, y)
        self.classes_ = np.unique(y)
        n_classes = len(self.classes_)

        if self.verbose:
            print(" >>> THE 21D SOPHISTICATED DIMENSIONALITY INITIATED <<<")
            print(" > Initiating The Ouroboros Protocol (Stabilized)...")

        # --- PHASE -1: THE UNIVERSAL LENS SELECTOR (Switching Scalers) ---
        # --- PHASE -1: THE UNIVERSAL LENS SELECTOR (Dual-Scout Protocol) ---
        if self.verbose: print(" > Phase -1: Selecting Universal Lens (Geometry + Logic Consensus)...")

        lenses = [
            ("Standard", StandardScaler()),
            ("Robust", RobustScaler(quantile_range=(15.0, 85.0))),
            ("MinMax", MinMaxScaler())
        ]

        best_lens_name = "Standard"
        best_lens_score = -1.0
        best_lens_obj = StandardScaler()

        # SCOUT TEAM: We use proxies for the two main laws of physics in HRF
        from sklearn.model_selection import cross_val_score
        from sklearn.tree import DecisionTreeClassifier

        # 1. Geometry Scout (Represents SVM, KNN, Soul, Gravity) -> Needs Scaling
        scout_geom = KNeighborsClassifier(n_neighbors=5, n_jobs=-1)

        # 2. Logic Scout (Represents ExtraTrees, XGBoost, Forest) -> Robust
        # We use a simple Tree to ensure the scaler doesn't distort the information gain.
        scout_logic = DecisionTreeClassifier(max_depth=5, random_state=42)

        # Test on subset (max 2000 samples for speed)
        sub_idx = np.random.choice(len(X), min(len(X), 2000), replace=False)
        X_sub = X[sub_idx]
        y_sub = y[sub_idx]

        for name, lens in lenses:
            try:
                # Apply Lens
                X_trans = lens.fit_transform(X_sub)

                # Get Consensus Score
                score_g = cross_val_score(scout_geom, X_trans, y_sub, cv=3, n_jobs=-1).mean()
                score_l = cross_val_score(scout_logic, X_trans, y_sub, cv=3, n_jobs=-1).mean()

                # Harmonic Mean (Penalizes if one scout hates it)
                # Formula: 2 * (G * L) / (G + L)
                combined_score = 2 * (score_g * score_l) / (score_g + score_l + 1e-9)

                if self.verbose:
                    print(f"    [{name:<8}] Geom: {score_g:.2%} | Logic: {score_l:.2%} | HARMONIC: {combined_score:.2%}")

                if combined_score > best_lens_score:
                    best_lens_score = combined_score
                    best_lens_name = name
                    best_lens_obj = lens
            except: pass

        self.scaler_ = best_lens_obj
        if self.verbose: print(f" >>> LENS LOCKED: {best_lens_name.upper()} SCALER (Consensus Achieved) <<<")

        X_scaled = self.scaler_.fit_transform(X)

        # --- PHASE 0: DUAL SNIPER CALIBRATION (Flash-Tune Protocol) ---
        if self.verbose: print(" > Phase 0: Calibrating Logic & Manifold Units (Flash-Tune)...")

        # [SPEED HACK]: We don't need 10k rows to find 'C'. 1000 is enough.
        # This makes it run 10x-50x faster.
        n_total = len(X)
        n_calib = min(n_total, 2000)

        # Stratified Subsample for Speed
        if n_total > 5000:
             # Fast random index
             idx_calib = np.random.choice(n_total, n_calib, replace=False)
             X_calib = X_scaled[idx_calib]
             y_calib = y[idx_calib]
        else:
             X_calib = X_scaled
             y_calib = y

        try:
            from sklearn.model_selection import RandomizedSearchCV

            # 1. Calibrate Resonance (Standard SVM)
            # Reduced iterations from 8 -> 5 (Good enough for coarse tuning)
            params_svc = {
                "C": [0.1, 1.0, 10.0, 50.0],
                "gamma": ["scale", "auto", 0.1]
            }
            search_svc = RandomizedSearchCV(
                self.unit_11, params_svc, n_iter=10, cv=3, n_jobs=-1, random_state=42
            )
            search_svc.fit(X_calib, y_calib)
            self.unit_11 = search_svc.best_estimator_

            # 2. Calibrate Nu-Warp (NuSVC)
            # Reduced iterations from 6 -> 4
            params_nu = {
                "nu": [0.05, 0.1, 0.2],
                "gamma": ["scale", "auto"]
            }
            search_nu = RandomizedSearchCV(
                self.unit_06, params_nu, n_iter=4, cv=3, n_jobs=-1, random_state=42
            )
            search_nu.fit(X_calib, y_calib)
            self.unit_06 = search_nu.best_estimator_

            if self.verbose:
                print(f"    >>> Resonance (SVM) Tuned: {search_svc.best_params_} | Score: {search_svc.best_score_:.2%}")
                print(f"    >>> Nu-Warp (NuSVC) Tuned: {search_nu.best_params_} | Score: {search_nu.best_score_:.2%}")
        except Exception as e:
            if self.verbose: print(f"    >>> Calibration Skipped (Speed Mode): {e}")

        # --- STEP 1: RAPID QUALIFIER (20% Proxy) ---
        X_train_sub, X_select, y_train_sub, y_select = train_test_split(
            X_scaled, y, test_size=0.20, stratify=y, random_state=42
        )

        # --- A: EVOLVE & TRAIN (On Sub-Set for Speed) ---
        if self.verbose:
            print(" > Phase 1: Awakening the Souls (Rapid Evolution)...")
            print("-" * 80)
            print(f" {'UNIT NAME':<20} | {'ACCURACY':<8} | {'EVOLVED DNA PARAMETERS'}")
            print("-" * 80)

        # 1. Define The Living Groups (Souls + Neural)
        # Note: We removed Cosmic/Forests from here to handle them in the Strict Order list below
        living_units = [
            ("SOUL-01 (Original)", self.unit_12),
            ("SOUL-02 (Mirror A)", self.unit_13),
            ("SOUL-03 (Mirror B)", self.unit_14),
            ("SOUL-D (AGI Hyper)", self.unit_15),
            ("SOUL-E (AGI Deep)", self.unit_16),
            ("SOUL-F (AGI Omni)", self.unit_17),
            ("NEURAL-ELM (Omni)", self.unit_25),
            # The Cosmic Forests (Now treated as Living Entities)
            ("GOLDEN-FOREST", self.unit_18),
            ("ENTROPY-FOREST", self.unit_19),
            ("QUANTUM-FOREST", self.unit_20),
            ("GRAVITY-FOREST", self.unit_21),
        ]

        # Evolve the Living
        for name, unit in living_units:
            if hasattr(unit, "set_raw_source"):
                unit.set_raw_source(X_train_sub)
            try:
                unit.fit(X_train_sub, y_train_sub)
                # Only evolve if supported
                if hasattr(unit, "evolve"):
                    acc = unit.evolve(X_select, y_select, generations=10)
                else:
                    acc = 0.0

                if self.verbose:
                    dna = getattr(unit, "dna_", {})
                    dna_str = "Standard"
                    # DNA Printer Logic
                    if "freq" in dna: dna_str = f"Freq: {dna['freq']:.2f} | Gamma: {dna['gamma']:.2f} | P: {dna.get('p', 2.0):.1f}"
                    elif "n_hidden" in dna: dna_str = f"H:{dna['n_hidden']} | Act:{dna['activation']} | Alpha:{dna['alpha']:.2f}"
                    # [NEW] Forest Printers
                    elif "resonance" in dna: dna_str = f"Res: {dna['resonance']:.3f} | Decay: {dna['decay']:.2f} | Shift: {dna['shift']:.1f}"
                    elif "horizon_pct" in dna: dna_str = f"Horizon: {dna['horizon_pct']}% | Power: {dna['decay_power']:.2f}"
                    elif "gamma" in dna and "n_components" in dna: dna_str = f"Gamma: {dna['gamma']:.2f} | N-Comp: {dna['n_components']}"
                    elif "n_components" in dna: dna_str = f"Components: {dna['n_components']}"

                    print(f" {name:<20} | {acc:.2%}  | {dna_str}")
            except Exception as e:
                if self.verbose: print(f" {name:<20} | FAILED   | {str(e)}")

        if self.verbose: print("-" * 80)

        # 2. Train The Non-Living (Standard + Competitors)
        # [TITAN UPDATE]: Removed Forests from here because they are now in the Living list above!
        non_living_training_group = [
            self.unit_01, self.unit_02, self.unit_03, self.unit_04, self.unit_05,
            self.unit_06, self.unit_07, self.unit_08, self.unit_09, self.unit_10, self.unit_11,
            # Forests removed from here
            self.unit_bench_svm, self.unit_bench_rf, self.unit_bench_xgb # Competitors
        ]

        for unit in non_living_training_group:
            try: unit.fit(X_train_sub, y_train_sub)
            except: pass

        # --- B: THE GRAND QUALIFIER (Identify Top 12) ---
        if self.verbose: print(" > Phase 2: The Grand Qualifier (Scanning All 12 Candidates)...")

        # CRITICAL: THIS ORDER MUST MATCH PREDICT_PROBA EXACTLY
        # 1. Standard (11)
        # 2. Cosmic (4)
        # 3. Competitors (3)
        # 4. Souls (6)
        # 5. Neural (1)
        all_units = [
            # 1. Standard
            self.unit_01, self.unit_02, self.unit_03, self.unit_04, self.unit_05,
            self.unit_06, self.unit_07, self.unit_08, self.unit_09, self.unit_10, self.unit_11,
            # 2. Cosmic / Physics
            self.unit_18, self.unit_19, self.unit_20, self.unit_21,
            # 3. Competitors
            self.unit_bench_svm, self.unit_bench_rf, self.unit_bench_xgb,
            # 4. Souls
            self.unit_12, self.unit_13, self.unit_14, self.unit_15, self.unit_16, self.unit_17,
            # 5. Neural
            self.unit_25,
            self.unit_26
        ]

        n_units = len(all_units)
        accs = []

        # Score all units on Selection Set
        for unit in all_units:
            try:
                p = unit.predict(X_select)
                accs.append(accuracy_score(y_select, p))
            except: accs.append(0.0)

        # Sort by raw accuracy
        sorted_indices = np.argsort(accs)[::-1]

        # [INSERT THIS SNIPPET IN PHASE 2 TO SEE ALL 21 SCORES]
        if self.verbose:
            print("\n" + "="*70)
            print(" >>> THE 21D PERFORMANCE MONITOR (Phase 2 Qualification) <<<")
            print("="*70)
            print(f" {'RANK':<6} | {'UNIT NAME':<18} | {'SCORE':<10} | {'STATUS'}")
            print("-" * 70)

            # Map indices to friendly names
            # (Indices 0-10 are Standard, 11+ are Living)
            # ... inside Phase 2 Performance Monitor ...
            # Map indices to friendly names
            # Order: Standard -> Fast Physics -> Benchmarks -> Souls -> Neural
            # Map indices to friendly names
            # Order: Standard -> Cosmic Forests -> Competitors -> Souls -> Neural
            map_names = [
                "Logic-ET", "Logic-RF", "Logic-HG", "Grad-XG1", "Grad-XG2",
                "Nu-Warp", "PolyKer", "Geom-K3", "Geom-K9", "Space-QDA", "Resonance",

                # [FIXED] Proper Names for the GPU Forests
                "GOLDEN-FOREST", "ENTROPY-FOREST", "QUANTUM-FOREST", "GRAVITY-FOREST",

                # Competitors
                "BENCH-SVM", "BENCH-RF", "BENCH-XGB",

                # Living Units
                "SOUL-Orig", "SOUL-TwinA", "SOUL-TwinB", "SOUL-D(AGI)", "SOUL-E(AGI)", "SOUL-F(AGI)",
                "Neural-ELM","THE DEATH RAY"
            ]

            for rank, idx in enumerate(sorted_indices):
                # Get Name
                if idx < len(map_names):
                    name = map_names[idx]
                else:
                    # Fallback if I missed an index
                    name = f"Unit-{idx}"

                score = accs[idx]
                status = "PROMOTED" if rank < 12 else "Eliminated"

                print(f" {rank+1:02d}     | {name:<18} | {score:.2%}    | {status}")
            print("-" * 70)

        # Pick Top 12 for the OOF Battle
        top_12_indices = sorted_indices[:12]
        candidate_models = [all_units[i] for i in top_12_indices]


        # --- C: THE OUROBOROS SELECTION (The Battle of Names) ---
        if self.verbose:
            print("\n" + "=" * 80)
            print(" >>> PHASE 3: THE OUROBOROS PROTOCOL (Top 12 Candidates - 100% Validation) <<<")
            print("=" * 80)
            print(f" {'RANK':<4} | {'UNIT NAME':<18} | {'OOF ACCURACY':<10} | {'STATUS'}")
            print("-" * 80)

        # 1. Define The Name Map (Global Index -> Name)
        # This matches your 21D init order perfectly.
        # 1. Define The Name Map (Global Index -> Name)
        all_names_map = [
            "Logic-ET", "Logic-RF", "Logic-HG", "Grad-XG1", "Grad-XG2",               # 0-4
            "Nu-Warp", "PolyKer", "Geom-K3", "Geom-K9", "Space-QDA", "Resonance",     # 5-10

            # [FIXED]
            "GOLDEN-FOREST", "ENTROPY-FOREST", "QUANTUM-FOREST", "GRAVITY-FOREST",    # 11-14

            "BENCH-SVM", "BENCH-RF", "BENCH-XGB",                                     # 15-17
            "SOUL-Orig", "SOUL-TwinA", "SOUL-TwinB", "SOUL-D(AGI)", "SOUL-E(AGI)", "SOUL-F(AGI)", # 18-23
            "Neural-ELM",                                                              # 24
            "THE DEATH RAY"
        ]

        candidate_oof_accs = []
        candidate_oof_preds_list = []

        # 2. Run OOF (With Real Names)
        for i, unit in enumerate(candidate_models):
            # Retrieve the Real Name using the index from Phase 2
            global_idx = top_12_indices[i]
            unit_name = all_names_map[global_idx] if global_idx < len(all_names_map) else f"Unit-{global_idx}"

            method = "predict_proba" if hasattr(unit, "predict_proba") else "decision_function"
            try:
                # 5-Fold Cross-Validation (The Truth Serum)
                oof_pred = cross_val_predict(unit, X_scaled, y, cv=5, method=method, n_jobs=-1)

                # Stabilization
                if method == "decision_function":
                    if len(oof_pred.shape) == 1:
                        p = 1 / (1 + np.exp(-oof_pred))
                        oof_pred = np.column_stack([1-p, p])
                    else:
                        max_d = np.max(oof_pred, axis=1, keepdims=True)
                        exp_d = np.exp(oof_pred - max_d)
                        oof_pred = exp_d / np.sum(exp_d, axis=1, keepdims=True)

                # Score
                acc_oof = accuracy_score(y, self.classes_[np.argmax(oof_pred, axis=1)])
                candidate_oof_accs.append(acc_oof)
                candidate_oof_preds_list.append(oof_pred)

                # Print The Battle Result
                if self.verbose:
                    print(f" {i+1:02d}   | {unit_name:<18} | {acc_oof:.4%}   | Validated")

            except Exception as e:
                candidate_oof_accs.append(0.0)
                candidate_oof_preds_list.append(np.zeros((len(X_scaled), len(self.classes_))))
                if self.verbose:
                    print(f" {i+1:02d}   | {unit_name:<18} | FAILED       | {str(e)[:20]}...")

        if self.verbose: print("-" * 80)

        # 3. Sort by Performance (Meritocracy)
        sorted_oof_idx = np.argsort(candidate_oof_accs)[::-1]

        # 4. Select Absolute Best (Top 2)


        # [TITAN SAFETY PROTOCOL: THE NEURAL LEASH]
        # Neural-ELM (Unit 25) is volatile. It must NEVER lead the Council.
        # If it wins Rank 1, we force-swap it with Rank 2.
        # This guarantees a stable model (Tree/SVM) always holds the 95%/85% power.

        # 4. Select Absolute Best (Top 2) - WITH TITAN SAFETY
        top_2_local_idx = []
        for idx in sorted_oof_idx:
            # Filter weak models
            if candidate_oof_accs[idx] < 0.10: continue

            # [TITAN SAFETY PROTOCOL: NEURAL EXILE]
            # Identify who this candidate is
            global_idx = top_12_indices[idx]

            # If it is the Neural-ELM (Index 24), we SKIP it.
            # This ensures it can never be Rank 1 or Rank 2.
            # It is effectively restricted to Rank 3 (Reserve Bench).
            if global_idx == 24:
                if self.verbose and len(top_2_local_idx) < 2:
                    print(f" > [SAFETY] Neural-ELM attempted to join Council. Request DENIED (Restricted to Rank 3).")
                continue

            top_2_local_idx.append(idx)
            if len(top_2_local_idx) == 2: break

        # Save The Elites (Now Safely Ordered)
        self.final_elites_ = [candidate_models[i] for i in top_2_local_idx]
        elite_accs = [candidate_oof_accs[i] for i in top_2_local_idx]
        elite_preds = [candidate_oof_preds_list[i] for i in top_2_local_idx]

        # --- ARCHITECTURE 1: THE COUNCIL  ---
        # Fixed 80/20 Split. Strong Leadership, but keeps a backup.
        self.weights_council_ = np.zeros(n_units)

        # Rank 1 gets 85%
        idx_rank1 = top_12_indices[top_2_local_idx[0]]
        self.weights_council_[idx_rank1] = 0.75

        # Rank 2 gets 15%
        idx_rank2 = top_12_indices[top_2_local_idx[1]]
        self.weights_council_[idx_rank2] = 0.25

        # --- ARCHITECTURE 2: THE ACE (Absolute Monarchy ) ---
        # The Winner takes ALL. Pure Power.
        self.weights_ace_ = np.zeros(n_units)
        self.weights_ace_[idx_rank1] = 0.90  # 95%
        self.weights_ace_[idx_rank2] = 0.10  # 5%

        # --- ARCHITECTURE 3: THE LINEAR (The Shield) ---
        # CRITICAL: Keep this 50/50.
        # If the Top Model is overfitting, this averages out the error.
        # This is your "Impossible to Lose" insurance policy.
        self.weights_linear_ = np.zeros(n_units)
        self.weights_linear_[idx_rank1] = 0.60
        self.weights_linear_[idx_rank2] = 0.40


        # --- ARCHITECTURE 4: THE BALANCE (Perfect Harmony 50/50) ---
        self.weights_balance_ = np.zeros(n_units)
        self.weights_balance_[idx_rank1] = 0.50
        self.weights_balance_[idx_rank2] = 0.50

        # --- ARCHITECTURE 5: THE INVERSION (Support Lead 40/60) ---
        self.weights_inv_linear_ = np.zeros(n_units)
        self.weights_inv_linear_[idx_rank1] = 0.40
        self.weights_inv_linear_[idx_rank2] = 0.60

        # --- ARCHITECTURE 6: THE UNDERDOG (Hidden Potential 30/70) ---
        self.weights_inv_council_ = np.zeros(n_units)
        self.weights_inv_council_[idx_rank1] = 0.30
        self.weights_inv_council_[idx_rank2] = 0.70

        # --- SIMULATION ---
        def get_score(weights_full):
            combined_pred = np.zeros_like(elite_preds[0])
            current_w = []
            for idx in top_2_local_idx:
                current_w.append(weights_full[top_12_indices[idx]])
            for i in range(2):
                combined_pred += current_w[i] * elite_preds[i]
            return accuracy_score(y, self.classes_[np.argmax(combined_pred, axis=1)])

        score_council = get_score(self.weights_council_)
        score_ace = get_score(self.weights_ace_)
        score_linear = get_score(self.weights_linear_)

        score_balance = get_score(self.weights_balance_)
        score_inv_linear = get_score(self.weights_inv_linear_)
        score_inv_council = get_score(self.weights_inv_council_)

        if self.verbose:
            print(f" > [STRATEGY LAB] Ace: {score_ace:.4%} | Council: {score_council:.4%} | Linear: {score_linear:.4%}")
            print(f" > [STRATEGY LAB] Balance: {score_balance:.4%} | Inv-Lin: {score_inv_linear:.4%} | Underdog: {score_inv_council:.4%}")

        # [TITAN 6-WAY TOURNAMENT]
        # We define a map of all 6 strategies and pick the absolute maximum
        strat_map = {
            "council": score_council,     # <--- Priority 2: The Vote (75/25)
            "linear": score_linear,       # <--- Priority 1: The Shield (60/40)
            "ace": score_ace,             # <--- Priority 4: The Gamble (90/10)
           # "balance": score_balance,     # <--- Priority 3: The Harmony (50/50)
            "inv_linear": score_inv_linear,
            "inv_council": score_inv_council
        }

        # Select the key with the highest value
        self.strategy_ = max(strat_map, key=strat_map.get)

        # [TITAN TIE-BREAKER PRESERVATION]
        # If Ace is essentially tied (>99% accuracy case), we still force Ace for purity
        if score_ace > 0.98 and abs(score_ace - strat_map[self.strategy_]) < 0.001:
            self.strategy_ = "ace"

        if self.verbose:
             print(f" >>> {self.strategy_.upper()} STRATEGY LOCKED. <<<")

        # --- PHASE 4: ASSIMILATION ---
        if self.verbose: print(f" > Phase 4: Final Assimilation (Retraining Top 2 Elites)...")
        for unit in self.final_elites_:
            unit.fit(X_scaled, y)

        # --- PHASE 4.5: THE DEATH RAY PROTOCOL (Corrected Logic) ---
        # 1. Get the Logic (OOF Probs) from the Rank 1 Elite
        rank1_local_idx = top_2_local_idx[0]
        best_oof_probs = elite_preds[0]

        # [CRITICAL FIX] Compare against the TRUE MAX of all 6 strategies
        # This prevents the Death Ray from beating a weak Ace but losing to a strong Balance.
        true_max_score = max(strat_map.values())
        true_max_name = max(strat_map, key=strat_map.get).upper()

        if self.verbose:
            print(f" > [HYPERNOVA] Elite Source Acquired: {candidate_models[rank1_local_idx].__class__.__name__}")
            print(f" > [HYPERNOVA] Current Champion to Beat: {true_max_score:.4%} ({true_max_name})")

        # 2. UNLEASH UNIT 26
        self.unit_26.fit_hunt(X_scaled, y, elite_probs_oof=best_oof_probs)

        # 3. VERIFY KILL
        death_ray_score = self.unit_26.verified_score_

        # MARGIN CHECK: Must beat the TRUE MAX score
        margin = death_ray_score - true_max_score

        if margin > 0.00001:
            if self.verbose:
                print(f" > [ALERT] DEATH RAY SUCCESSFUL. (Score: {death_ray_score:.4%} | Margin: +{margin:.4%})")
                print(f" > [COMMAND] OVERRIDING STRATEGY -> DEATH_RAY.")

            self.strategy_ = "death_ray"
            self.weights_death_ray_ = np.zeros(26)

            # 60% Death Ray + 40% Elite Foundation
            self.weights_death_ray_[25] = 0.05
            rank1_global_idx = top_12_indices[rank1_local_idx]
            self.weights_death_ray_[rank1_global_idx] = 0.95

        else:
             if self.verbose:
                 print(f" > [DEATH RAY] Stand Down. No gain over {true_max_name} (Ray: {death_ray_score:.4%} vs Champ: {true_max_score:.4%}).")
             self.weights_death_ray_ = np.zeros(26)


        # --- PHASE 5: THE CHRONOS GATE ASSIMILATION (Dynamic Constitution) ---
        if self.verbose:
            print("\n" + "="*85)
            print(f" >>> PHASE 5: THE CHRONOS GATE ASSIMILATION (Training the Router) <<<")
            print("="*85)

        # 1. Reconstruct Strategy OOF Predictions
        # We need to know what each strategy WOULD have predicted for the training data.
        
        # Helper to mix OOFs based on strategy weights
        def get_oof_strat(weights_full):
            combined = np.zeros_like(elite_preds[0])
            w1 = weights_full[top_12_indices[top_2_local_idx[0]]] # Rank 1 Weight
            w2 = weights_full[top_12_indices[top_2_local_idx[1]]] # Rank 2 Weight
            
            if w1 > 0: combined += w1 * elite_preds[0]
            if w2 > 0: combined += w2 * elite_preds[1]
            
            sum_w = w1 + w2
            if sum_w > 0: return combined / sum_w
            return combined

        # Generate OOF matrices for each candidate strategy
        oof_council = get_oof_strat(self.weights_council_)
        oof_ace     = get_oof_strat(self.weights_ace_)
        oof_linear  = get_oof_strat(self.weights_linear_)
        
        # For Death Ray OOF, use the fit_hunt result directly if active
        if self.unit_26.verified_score_ > 0:
            # Reconstruct the Death Ray's view using Rank 1 as base
            oof_death_ray = self.unit_26.predict_proba(
                np.hstack([X_scaled, elite_preds[0]]) 
            )
        else:
            oof_death_ray = oof_council # Fallback if Ray failed

        # 2. Determine "Who Won?" for each sample
        # The Strategy with the HIGHEST confidence in the RIGHT answer wins that sample.
        n_samples = len(y)
        strat_scores = np.zeros((n_samples, 4)) # [Council, Ace, Linear, Ray]
        
        # Get true class indices
        true_class_idx = np.searchsorted(self.classes_, y)
        
        # Extract prob of true class
        row_idx = np.arange(n_samples)
        strat_scores[:, 0] = oof_council[row_idx, true_class_idx]
        strat_scores[:, 1] = oof_ace[row_idx, true_class_idx]
        strat_scores[:, 2] = oof_linear[row_idx, true_class_idx]
        strat_scores[:, 3] = oof_death_ray[row_idx, true_class_idx]
        
        # Select best strategy per sample
        best_strat_indices = np.argmax(strat_scores, axis=1)
        
        # 3. Train The Gate
        if self.verbose:
            counts = np.bincount(best_strat_indices, minlength=4)
            print(f" > [CHRONOS] Strategy Wins in Training:")
            print(f"   - Council:   {counts[0]} samples")
            print(f"   - Ace:       {counts[1]} samples")
            print(f"   - Linear:    {counts[2]} samples")
            print(f"   - Death Ray: {counts[3]} samples")
            print(f" > [CHRONOS] Training Gatekeeper on GPU T4...")

        self.unit_chronos.fit(X_scaled, best_strat_indices)
        self.strategy_ = "chronos_dynamic" # Flag to use the gate in predict
        
        if self.verbose:
             print(" > [CHRONOS] Gatekeeper Active. Constitution is now LIVING.")
             print("-" * 85)

        # --- FINAL CALCULATION: DUAL MARGIN CHECK ---
        if X_test_oracle is not None and y_test_oracle is not None:
            # 1. Benchmark
            X_bench = self.scaler_.transform(X_test_oracle)
            score_bench = accuracy_score(y_test_oracle, self.unit_bench_xgb.predict(X_bench))

            # 2. Helper
            def manual_score(weights):
                if np.sum(weights) == 0: return 0.0
                final_p = np.zeros((len(X_test_oracle), len(self.classes_)))
                X_test_sc = self.scaler_.transform(X_test_oracle)
                rank1_idx = top_12_indices[top_2_local_idx[0]]
                elite_p = all_units_ordered[rank1_idx].predict_proba(X_test_sc)

                for i, w in enumerate(weights):
                    if w > 0:
                        unit = all_units_ordered[i]
                        try:
                            if i == 25: # Death Ray
                                X_fused = np.hstack([X_test_sc, elite_p])
                                p = unit.predict_proba(X_fused)
                            elif i == rank1_idx: # Opt
                                p = elite_p
                            else:
                                if hasattr(unit, "predict_proba"): p = unit.predict_proba(X_test_sc)
                                else:
                                     d = unit.decision_function(X_test_sc)
                                     p = np.exp(d)/np.sum(np.exp(d), axis=1, keepdims=True)
                            final_p += w * p
                        except: pass
                return accuracy_score(y_test_oracle, self.classes_[np.argmax(final_p, axis=1)])

            # 3. Score Reality A (Standard)
            w_map = {
                "ACE": self.weights_ace_, "COUNCIL": self.weights_council_,
                "LINEAR": self.weights_linear_, "BALANCE": self.weights_balance_,
                "INV_LINEAR": self.weights_inv_linear_, "INV_COUNCIL": self.weights_inv_council_
            }
            score_champ = manual_score(w_map.get(std_name, self.weights_council_))
            margin_champ = score_champ - score_bench

            # 4. Score Reality B (Death Ray)
            score_ray = 0.0
            if hasattr(self, "weights_death_ray_") and np.sum(self.weights_death_ray_) > 0:
                score_ray = manual_score(self.weights_death_ray_)

            # Internal Boost Calculation
            boost = score_ray - score_champ

            # Print Final Report
            print("-" * 85)
            print(f" [BENCHMARK] XGBoost Baseline        | {score_bench:.4%}    | Target to Beat")
            print("-" * 85)
            print(f" HRF Ultimate (Standard: {std_name:<7}) | {score_champ:.4%}    | Margin: {margin_champ:+.4%}")

            if score_ray > 0:
                ray_result_status = f"Margin: {score_ray - score_bench:+.4%} (Boost: {boost:+.4%})"
                print(f" HRF Ultimate (The Death Ray)    | {score_ray:.4%}    | {ray_result_status}")
            else:
                print(f" HRF Ultimate (The Death Ray)    | INACTIVE        | (Ray did not fire)")

            print("-" * 85)

        return self


    def _predict_council_internal(self, X):
        # Fast prediction using pre-calculated weights
        X_sc = self.scaler_.transform(X)
        final_pred = None
        all_units= [
            # 1. Logic (01-11)
            self.unit_01, self.unit_02, self.unit_03, self.unit_04, self.unit_05,
            self.unit_06, self.unit_07, self.unit_08, self.unit_09, self.unit_10, self.unit_11,

            # 2. THE NEW GPU FORESTS (18-21)
            self.unit_18, self.unit_19, self.unit_20, self.unit_21,

            # 3. Competitors
            self.unit_bench_svm, self.unit_bench_rf, self.unit_bench_xgb,

            # 4. Souls (12-17)
            self.unit_12, self.unit_13, self.unit_14, self.unit_15, self.unit_16, self.unit_17,

            # 5. Neural (25)
            self.unit_25,
            self.unit_26
        ]
        # ... inside the loop where you iterate over units ...
        for i, unit in enumerate(all_units):
            if active_weights[i] > 0:
                try:
                    # SPECIAL HANDLING FOR UNIT 26 (HYPERNOVA)
                    if i == 25: # Unit 26 is index 25 in the all_units list
                        # We need the Rank 1 Elite's prediction on this NEW data
                        # Recalculate Rank 1 Elite (It's already in self.final_elites_[0])
                        # This works because Phase 4 retrained them on all data.
                        elite_p = self.final_elites_[0].predict_proba(X_scaled)

                        # Fuse manually for prediction
                        X_fused_new = np.hstack([X_scaled, elite_p])

                        # Unit 26 predicts on Fused Data
                        p = unit.predict_proba(X_fused_new)

                    else:
                        # Standard Units
                        if hasattr(unit, "predict_proba"):
                            p = unit.predict_proba(X_scaled)
                        else:
                            d = unit.decision_function(X_scaled)
                            max_d = np.max(d, axis=1, keepdims=True)
                            exp_d = np.exp(d - max_d)
                            p = exp_d / np.sum(exp_d, axis=1, keepdims=True)

                    # Accumulate
                    if final_pred is None: final_pred = active_weights[i] * p
                    else: final_pred += active_weights[i] * p
                except: pass


        if final_pred is None: return np.zeros(len(X)) # Fallback
        return self.classes_[np.argmax(final_pred, axis=1)]


    def _predict_proba_council_internal(self, X_scaled):
        """
        Calculates the weighted probability matrix of the Council.
        Essential for the Alien-Z sharpening process.
        """
        final_pred = None
        # Must match the order in your init
        all_units= [
            # 1. Logic (01-11)
            self.unit_01, self.unit_02, self.unit_03, self.unit_04, self.unit_05,
            self.unit_06, self.unit_07, self.unit_08, self.unit_09, self.unit_10, self.unit_11,

            # 2. THE NEW GPU FORESTS (18-21)
            self.unit_18, self.unit_19, self.unit_20, self.unit_21,

            # 3. Competitors
            self.unit_bench_svm, self.unit_bench_rf, self.unit_bench_xgb,

            # 4. Souls (12-17)
            self.unit_12, self.unit_13, self.unit_14, self.unit_15, self.unit_16, self.unit_17,

            # 5. Neural (25)
            self.unit_25,
            self.unit_26
        ]

        # ... inside the loop where you iterate over units ...
        for i, unit in enumerate(all_units):
            if active_weights[i] > 0:
                try:
                    # SPECIAL HANDLING FOR UNIT 26 (HYPERNOVA)
                    if i == 25: # Unit 26 is index 25 in the all_units list
                        # We need the Rank 1 Elite's prediction on this NEW data
                        # Recalculate Rank 1 Elite (It's already in self.final_elites_[0])
                        # This works because Phase 4 retrained them on all data.
                        elite_p = self.final_elites_[0].predict_proba(X_scaled)

                        # Fuse manually for prediction
                        X_fused_new = np.hstack([X_scaled, elite_p])

                        # Unit 26 predicts on Fused Data
                        p = unit.predict_proba(X_fused_new)

                    else:
                        # Standard Units
                        if hasattr(unit, "predict_proba"):
                            p = unit.predict_proba(X_scaled)
                        else:
                            d = unit.decision_function(X_scaled)
                            max_d = np.max(d, axis=1, keepdims=True)
                            exp_d = np.exp(d - max_d)
                            p = exp_d / np.sum(exp_d, axis=1, keepdims=True)

                    # Accumulate
                    if final_pred is None: final_pred = active_weights[i] * p
                    else: final_pred += active_weights[i] * p
                except: pass

        # Safety fallback
        if final_pred is None:
            return np.ones((len(X_scaled), len(self.classes_))) / len(self.classes_)

        # Normalize to ensure sum=1.0
        return final_pred / np.sum(final_pred, axis=1, keepdims=True)

    def _get_stack_features(self, X_scaled):
        """
        Helper to gather predictions for the Linear Mirror strategy.
        """
        X_stack_list = []
        for unit in self.final_elites_:
            if hasattr(unit, "predict_proba"):
                p = unit.predict_proba(X_scaled)
            else:
                d = unit.decision_function(X_scaled)
                p = np.exp(d) / np.sum(np.exp(d), axis=1, keepdims=True)
            X_stack_list.append(p)
        return np.hstack(X_stack_list)


    def _predict_mirror_internal(self, X, mode="hybrid"):
        X_sc = self.scaler_.transform(X)
        X_stack_list = []
        for unit in self.final_elites_:
            if hasattr(unit, "predict_proba"): p = unit.predict_proba(X_sc)
            else:
                d = unit.decision_function(X_sc)
                p = np.exp(d) / np.sum(np.exp(d), axis=1, keepdims=True)
            X_stack_list.append(p)
        X_stack = np.hstack(X_stack_list)

        model = self.unit_mirror_hybrid if mode == "hybrid" else self.unit_mirror_linear
        return model.predict(X_stack)


    def predict_proba(self, X):
        X_scaled = self.scaler_.transform(X)

        # --- PREPARE STRATEGY BASES (The Ingredients) ---
        # 1. Get Top 2 Elite Predictions
        pred_rank1 = self.final_elites_[0].predict_proba(X_scaled)
        pred_rank2 = self.final_elites_[1].predict_proba(X_scaled)
        
        # 2. Construct The Strategies
        p_council = (0.75 * pred_rank1) + (0.25 * pred_rank2)
        p_ace = (0.90 * pred_rank1) + (0.10 * pred_rank2)
        p_linear = (0.60 * pred_rank1) + (0.40 * pred_rank2)
        
        # Death Ray (Requires fusion)
        X_fused = np.hstack([X_scaled, pred_rank1])
        p_death_ray = self.unit_26.predict_proba(X_fused)
        
        # --- APPLY CHRONOS GATE (The Router) ---
        # Get dynamic weights: [w_council, w_ace, w_linear, w_ray]
        gate_weights = self.unit_chronos.get_gate_weights(X_scaled)
        
        # --- FINAL MIXING ---
        final_pred = (
            gate_weights[:, 0:1] * p_council +
            gate_weights[:, 1:2] * p_ace +
            gate_weights[:, 2:3] * p_linear +
            gate_weights[:, 3:4] * p_death_ray
        )
        
        # Normalize
        row_sums = final_pred.sum(axis=1, keepdims=True)
        return final_pred / (row_sums + 1e-9)



    def predict(self, X):
        return self.classes_[np.argmax(self.predict_proba(X), axis=1)]


def HarmonicResonanceForest_Ultimate(n_estimators=None):
    return HarmonicResonanceClassifier_BEAST_21D(verbose=True)

# --- ADD THIS AT THE ABSOLUTE BOTTOM ---
if __name__ == "__main__":
    # 1. Put your data loading here
    # X, y = load_your_data()

    # 2. Put your model execution here
    # model = HarmonicResonanceForest_Ultimate()
    # model.fit(X, y)

    print("✅ Titan-21 Safety Protocol Engaged. System is stable.")

✅ GPU DETECTED: HRF v26.0 'Holo-Fractal Universe' Active
✅ Titan-21 Safety Protocol Engaged. System is stable.


In [2]:
from sklearn.datasets import fetch_openml
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.utils import resample
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer

# Updated to accept custom_X and custom_y
def run_comparative_benchmark(dataset_name, openml_id, sample_limit=3000, custom_X=None, custom_y=None):
    print(f"\n[DATASET] Loading {dataset_name} (ID: {openml_id})...")

    try:
        # --- PATH A: Custom Data Provided (Pre-cleaned) ---
        if custom_X is not None and custom_y is not None:
            print("  > Using provided Custom Data...")
            X = custom_X
            y = custom_y

            # Ensure X is numpy (in case a DF was passed)
            if hasattr(X, 'values'):
                X = X.values

        # --- PATH B: Fetch from OpenML ---
        else:
            # Fetch as DataFrame to handle types better
            X_df, y = fetch_openml(data_id=openml_id, return_X_y=True, as_frame=True, parser='auto')

            # 1. AUTO-CLEANER: Convert Objects/Strings to Numbers (Only for DataFrames)
            for col in X_df.columns:
                if X_df[col].dtype == 'object' or X_df[col].dtype.name == 'category':
                    le = LabelEncoder()
                    X_df[col] = le.fit_transform(X_df[col].astype(str))

            X = X_df.values # Convert to Numpy for HRF

        # --- COMMON PIPELINE (NaN Handling) ---
        # Even if custom data is passed, we double-check for NaNs to be safe
        if np.isnan(X).any():
            print("  > NaNs detected. Imputing with Mean strategy...")
            imp = SimpleImputer(strategy='mean')
            X = imp.fit_transform(X)

        le_y = LabelEncoder()
        y = le_y.fit_transform(y)

        # 3. GPU Limit Check
        if len(X) > sample_limit:
            print(f"  ...Downsampling from {len(X)} to {sample_limit} (GPU Limit)...")
            X, y = resample(X, y, n_samples=sample_limit, random_state=42, stratify=y)

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42, stratify=y)
        print(f"  Shape: {X.shape} | Classes: {len(np.unique(y))}")

    except Exception as e:
        print(f"  Error loading data: {e}")
        return

    competitors = {
        "SVM (RBF)": make_pipeline(StandardScaler(), SVC(kernel='rbf', C=1.0, probability=True, random_state=42)),
        "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1),
        "XGBoost (GPU)": XGBClassifier(
            device='cuda',
            tree_method='hist',
            #use_label_encoder=False,
            eval_metric='logloss',
            random_state=42
        ),
        # Ensure your HRF class is defined in the notebook before running this
        "HRF Ultimate (GPU)": HarmonicResonanceForest_Ultimate(n_estimators=60)
    }

    results = {}
    print(f"\n[BENCHMARK] Executing comparisons on {dataset_name}...")
    print("-" * 65)
    print(f"{'Model Name':<25} | {'Accuracy':<10} | {'Status'}")
    print("-" * 65)

    hrf_acc = 0

    for name, model in competitors.items():
        try:
            model.fit(X_train, y_train)
            preds = model.predict(X_test)
            acc = accuracy_score(y_test, preds)
            results[name] = acc
            print(f"{name:<25} | {acc:.4%}    | Done")

            if "HRF" in name:
                hrf_acc = acc

        except Exception as e:
            print(f"{name:<25} | FAILED      | {e}")

    print("-" * 65)

    best_competitor = 0
    for k, v in results.items():
        if "HRF" not in k and v > best_competitor:
            best_competitor = v

    margin = hrf_acc - best_competitor

    if margin > 0:
        print(f" HRF WINNING MARGIN: +{margin:.4%}")
    else:
        print(f" HRF GAP: {margin:.4%}")

In [5]:
# TEST 2: Phoneme (Search by Name to bypass ID error)
run_comparative_benchmark(
    dataset_name="Phoneme",
    openml_id=None,  # We will skip ID
    sample_limit=5404 
)


[DATASET] Loading Phoneme (ID: None)...
  Error loading data: Neither name nor data_id are provided. Please provide name or data_id.
